In [1]:
#import json
import ast

#Find the SCC around a given pivot state
#by finding an intersection of Forwards and Backwards reachability from pivot

def intersect_FB( visitedF, visitedB ):
    result = {} #will be stored as dictionary {rectangle : {entryF facet : list exitF facets}} in the forward direction
    for rkey in visitedF:
        if rkey in visitedB:
            ffacets = visitedF[ rkey ] # entry-> exits forward
            bfacets = visitedB[ rkey ] # exit -> entries from fw point of view
            rresult = { }# facet:list of exits for this given rectangle
            #compare, store the "intersection"
            r_intersectionF = {} #in the form of entry facet F : list of exit facets F
            for fkey in ffacets:
                #if fkey somewhere in list of some bfkey
                for bfkey in ffacets[ fkey ]: #forward [] of exits
                    if bfkey in bfacets: #is exit listed in bacwards?
                        if fkey in bfacets[ bfkey ]: #does exit backwards lead to fw entry?
                            #store
                            if fkey not in r_intersectionF:
                                r_intersectionF[ fkey ] = []
                            r_intersectionF[ fkey ].append( bfkey )
            result[ rkey ] = r_intersectionF #for rectangle r the intersection of fw and bw flow between facets
    return result

def filter_with_entry_exit_pairs( dictionary ):
    result = {}
    for key in dictionary:
        if dictionary[ key ] != {}:
            result[ key ] = dictionary[ key ]
    return result

def write_out_inter( dictionar, i ):
    #write content to the output file
    fo = open("./outVDPinter/output"+str(i)+".txt", "w")
    #fo.write("Intersection of forward and backward reachable states:\n")
    fo.write( str(dictionar) )
    fo.close()

def read_dict_from_file( fb, i ):
    if fb == "f":
        fo = open("./outVDPforward/output"+str(i)+".txt", "r")
    else:
        fo = open("./outVDPbackward/output"+str(i)+".txt", "r")
    data = fo.read()
    fo.close()
    #js = json.loads(data)
    #with_double_quotes = data.replace( "\'", '"' )
    dictionary = ast.literal_eval( data ) #with_double_quotes )
    return dictionary


In [2]:
#Read forward and backward reachable sets for
#Intersect for rectangles (entry [] the whole rectangle) in init
#write out the results - name them with identifiers "_x_y"
#initial_rectangles = []
identifiers = []
for x in [-3, -2, -1, 0, 1, 2, 3]:
    for y in [-3, -2, -1, 0, 1, 2, 3]:
#        initial_rectangles.append( find_rectangle( [ x, y ] ) )
        identifiers.append( "_" + str( x ) + "_" + str( y ) )

F_visited_sets = {}
F_union_visited = {}
B_visited_sets = {}
B_union_visited = {}

#for r,i in zip( initial_rectangles, identifiers ):
for i in identifiers:
    visitedF = read_dict_from_file( "f", "Konec"+i )
    F_visited_sets[ i ] = visitedF
    F_union_visited |= visitedF
    
    visitedB = read_dict_from_file( "b", "Konec"+i )
    B_visited_sets[ i ] = visitedB
    B_union_visited |= visitedB
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    print( intersect )
    #add initial rectangles?

write_out_inter( F_union_visited, "UnionForward" )
write_out_inter( B_union_visited, "UnionBackward" )


{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}


In [3]:
#visualize the sets, write png images
#need the tresholds and rectangle->geom real
#import
import numpy as np
import seaborn as sns
# Matplotlib for additional customization
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd

def r_hash( h ):
    r = [0,0] #default value
    hparts = h.split('|')
    r[0] = int( hparts[0] )
    r[1] = int( hparts[1] )
    return r

#model-specific tresholds
x0min = -8
x0max = 8
x1min = -8
x1max = 8

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

def visualize_rectangles( rdict, i ):
    list_r = []
    
    for key in rdict:
        r = r_hash( key )
        list_r.append( r )
    
    xlist = []
    ylist = []
    for r in list_r:
        xlist.append( 0.5*( tresholds[0][r[0]] + tresholds[0][r[0]+1]) )
        ylist.append( 0.5*( tresholds[1][r[1]] + tresholds[1][r[1]+1]) )

    ctverecky = pd.DataFrame({
        "x" : xlist,
        "y" : ylist
    })

    wid = 5
    hei = 5
    fig,ax = plt.subplots(1, 1, figsize=(wid, hei))
    sns.scatterplot(x="x", y="y", data=ctverecky, markers="s", s=10)
    ax.set(xlim=(x0min, x0max))
    ax.set(ylim=(x1min, x1max))
    plt.savefig('./outVDPinter/img'+str(i)+'.png', dpi=300)
    plt.clf()
    plt.close()

for i in identifiers:
    visitedF = F_visited_sets[ i ]
    visualize_rectangles( visitedF, "Forw"+i )
    visitedB = B_visited_sets[ i ]
    visualize_rectangles( visitedF, "Backw"+i )
        
    intersect = intersect_FB( visitedF, visitedB )
    write_out_inter( intersect, i )
    intersect = filter_with_entry_exit_pairs( intersect )
    visualize_rectangles( intersect, "Intersection"+i )
    #add initial rectangles?
    
visualize_rectangles( F_union_visited, "UnionForward" )
visualize_rectangles( B_union_visited, "UnionBackward" )


In [4]:
#older experiments

#visitedF = {'43|37': {'01': ['00', '11']}, '43|38': {'10': ['00']}, '44|38': {'01': ['00', '10'], '10': ['00']}, '44|39': {'11': ['11']}, '44|40': {'10': ['00']}, '45|40': {'01': ['00', '10']}, '45|41': {'11': ['00']}, '46|41': {'01': ['10'], '11': ['00']}, '46|42': {'11': ['00']}, '47|42': {'01': ['10'], '11': ['00', '10']}, '47|43': {'11': ['00', '10']}, '47|44': {'11': ['00']}, '48|44': {'01': ['10'], '11': ['10', '00']}, '48|45': {'11': ['00', '10']}, '48|46': {'11': ['00']}, '49|46': {'01': ['10'], '11': ['00', '10']}, '49|47': {'11': ['00', '10']}, '49|48': {'11': ['00']}, '50|48': {'01': ['10'], '11': ['10', '00']}, '50|49': {'11': ['10', '00']}, '50|50': {'11': []}, '50|47': {'01': ['10'], '11': ['00', '10']}, '51|49': {'01': ['10'], '11': ['10', '00']}, '51|50': {'11': ['01']}, '52|50': {'00': ['10'], '11': ['01', '10']}, '52|51': {'11': []}, '49|45': {'01': ['10'], '11': ['00', '10']}, '51|48': {'01': ['10'], '11': ['10', '00']}, '50|46': {'01': ['10'], '11': ['00', '10']}, '51|47': {'01': ['10'], '11': ['10', '00']}, '52|49': {'01': ['10'], '11': ['10', '00']}, '53|50': {'00': ['10'], '11': ['10', '01']}, '53|51': {'11': ['01']}, '54|51': {'00': ['10'], '11': ['10']}, '54|52': {'11': ['01']}, '55|52': {'00': ['10']}, '55|53': {'11': ['01', '10']}, '55|54': {'11': ['01']}, '56|54': {'00': ['10'], '11': ['01', '10']}, '56|55': {'11': ['01']}, '57|55': {'00': ['10'], '11': ['10']}, '57|56': {'11': ['10', '01']}, '57|57': {'11': ['01', '10']}, '57|58': {'11': ['01']}, '58|58': {'00': ['10'], '11': ['10', '01']}, '58|59': {'11': ['10', '01']}, '58|60': {'11': ['01', '10']}, '58|61': {'11': ['01']}, '59|61': {'00': ['10'], '11': ['10', '01']}, '59|62': {'11': ['10', '01']}, '59|63': {'11': ['01', '10']}, '59|64': {'11': ['01', '10']}, '59|65': {'11': ['01', '10']}, '59|66': {'11': ['01', '10']}, '59|67': {'11': ['01', '10']}, '59|68': {'11': ['01', '10']}, '59|69': {'11': ['01', '10']}, '59|70': {'11': ['01', '10']}, '59|71': {'11': ['01', '10']}, '59|72': {'11': ['01', '10']}, '59|73': {'11': ['01', '10']}, '59|74': {'11': ['01', '10']}, '59|75': {'11': ['01', '10']}, '59|76': {'11': ['01', '10']}, '59|77': {'11': ['01', '10']}, '59|78': {'11': ['01', '10']}, '59|79': {'11': ['01', '10']}, '59|80': {'11': ['01', '10']}, '59|81': {'11': ['01', '10']}, '59|82': {'11': ['01', '10']}, '59|83': {'11': ['01', '10']}, '59|84': {'11': ['01', '10']}, '59|85': {'11': ['01', '10']}, '59|86': {'11': ['01', '10']}, '59|87': {'11': ['01', '10']}, '59|88': {'11': ['01', '10']}, '59|89': {'11': ['01', '10']}, '59|90': {'11': ['01', '10']}, '59|91': {'11': ['01', '10']}, '59|92': {'11': ['01', '10']}, '59|93': {'11': ['01', '10']}, '59|94': {'11': ['01', '10']}, '59|95': {'11': ['01', '10']}, '59|96': {'11': ['01', '10']}, '59|97': {'11': ['01', '10']}, '59|98': {'11': ['01', '10']}, '59|99': {'11': ['01', '10']}, '60|99': {'00': ['10'], '11': ['10', '01']}, '60|98': {'00': ['10'], '11': ['10', '01']}, '60|97': {'00': ['10'], '11': ['10', '01']}, '61|99': {'00': ['10'], '11': ['10']}, '60|96': {'00': ['10'], '11': ['10', '01']}, '61|98': {'00': ['10'], '11': ['10']}, '60|95': {'00': ['10'], '11': ['10', '01']}, '61|97': {'00': ['10'], '11': ['10']}, '60|94': {'00': ['10'], '11': ['10', '01']}, '61|96': {'00': ['10'], '11': ['10']}, '60|93': {'00': ['10'], '11': ['10', '01']}, '61|95': {'00': ['10'], '11': ['10', '01']}, '60|92': {'00': ['10'], '11': ['10', '01']}, '61|94': {'00': ['10'], '11': ['10', '01']}, '60|91': {'00': ['10'], '11': ['10', '01']}, '61|93': {'00': ['10'], '11': ['10', '01']}, '62|95': {'00': ['10'], '11': ['10']}, '62|96': {'11': ['10']}, '62|97': {'11': ['10']}, '62|98': {'11': ['10']}, '62|99': {'11': ['10']}, '60|90': {'00': ['10'], '11': ['10', '01']}, '61|92': {'00': ['10'], '11': ['10', '01']}, '62|94': {'00': ['10'], '11': ['10']}, '60|89': {'00': ['10'], '11': ['10', '01']}, '61|91': {'00': ['10'], '11': ['10', '01']}, '62|93': {'00': ['10'], '11': ['10']}, '60|88': {'00': ['10'], '11': ['10', '01']}, '61|90': {'00': ['10'], '11': ['10', '01']}, '62|92': {'00': ['10'], '11': ['10']}, '60|87': {'00': ['10'], '11': ['10', '01']}, '61|89': {'00': ['10'], '11': ['10', '01']}, '62|91': {'00': ['10'], '11': ['10']}, '60|86': {'00': ['10'], '11': ['10', '01']}, '61|88': {'00': ['10'], '11': ['10', '01']}, '62|90': {'00': ['10'], '11': ['10']}, '60|85': {'00': ['10'], '11': ['10', '01']}, '61|87': {'00': ['10'], '11': ['10', '01']}, '62|89': {'00': ['10'], '11': ['10']}, '60|84': {'00': ['10'], '11': ['10', '01']}, '61|86': {'00': ['10'], '11': ['10', '01']}, '62|88': {'00': ['10'], '11': ['10']}, '60|83': {'00': ['10'], '11': ['10', '01']}, '61|85': {'00': ['10'], '11': ['10', '01']}, '62|87': {'00': ['10'], '11': ['10']}, '60|82': {'00': ['10'], '11': ['10', '01']}, '61|84': {'00': ['10'], '11': ['10', '01']}, '62|86': {'00': ['10'], '11': ['10']}, '60|81': {'00': ['10'], '11': ['10', '01']}, '61|83': {'00': ['10'], '11': ['10', '01']}, '62|85': {'00': ['10'], '11': ['10']}, '60|80': {'00': ['10'], '11': ['10', '01']}, '61|82': {'00': ['10'], '11': ['10', '01']}, '62|84': {'00': ['10'], '11': ['10']}, '60|79': {'00': ['10'], '11': ['10', '01']}, '61|81': {'00': ['10'], '11': ['10', '01']}, '62|83': {'00': ['10'], '11': ['10']}, '60|78': {'00': ['10'], '11': ['10', '01']}, '61|80': {'00': ['10'], '11': ['10', '01']}, '62|82': {'00': ['10'], '11': ['10']}, '60|77': {'00': ['10'], '11': ['10', '01']}, '61|79': {'00': ['10'], '11': ['10', '01']}, '62|81': {'00': ['10'], '11': ['10']}, '60|76': {'00': ['10'], '11': ['10', '01']}, '61|78': {'00': ['10'], '11': ['10', '01']}, '62|80': {'00': ['10'], '11': ['10']}, '60|75': {'00': ['10'], '11': ['10', '01']}, '61|77': {'00': ['10'], '11': ['10', '01']}, '62|79': {'00': ['10'], '11': ['10']}, '60|74': {'00': ['10'], '11': ['10', '01']}, '61|76': {'00': ['10'], '11': ['10', '01']}, '62|78': {'00': ['10'], '11': ['10']}, '60|73': {'00': ['10'], '11': ['10', '01']}, '61|75': {'00': ['10'], '11': ['10', '01']}, '62|77': {'00': ['10'], '11': ['10']}, '60|72': {'00': ['10'], '11': ['10', '01']}, '61|74': {'00': ['10'], '11': ['10', '01']}, '62|76': {'00': ['10'], '11': ['10']}, '60|71': {'00': ['10'], '11': ['10', '01']}, '61|73': {'00': ['10'], '11': ['10', '01']}, '62|75': {'00': ['10'], '11': ['10']}, '60|70': {'00': ['10'], '11': ['10', '01']}, '61|72': {'00': ['10'], '11': ['10', '01']}, '62|74': {'00': ['10'], '11': ['10']}, '60|69': {'00': ['10'], '11': ['10', '01']}, '61|71': {'00': ['10'], '11': ['10']}, '62|73': {'00': ['10'], '11': ['10']}, '60|68': {'00': ['10'], '11': ['10', '01']}, '61|70': {'00': ['10'], '11': ['10']}, '62|72': {'00': ['10']}, '60|67': {'00': ['10'], '11': ['10', '01']}, '61|69': {'00': ['10'], '11': ['10']}, '60|66': {'00': ['10'], '11': ['10', '01']}, '61|68': {'00': ['10'], '11': ['10']}, '60|65': {'00': ['10'], '11': ['10', '01']}, '61|67': {'00': ['10'], '11': ['10']}, '60|64': {'00': ['10'], '11': ['10', '01']}, '61|66': {'00': ['10'], '11': ['10']}, '60|63': {'00': ['10'], '11': ['10', '01']}, '61|65': {'00': ['10'], '11': ['10']}, '59|60': {'00': ['10'], '11': ['10', '01']}, '60|62': {'00': ['10'], '11': ['10', '01']}, '61|64': {'00': ['10'], '11': ['10']}, '58|57': {'00': ['10'], '11': ['10']}, '59|59': {'00': ['10'], '11': ['10']}, '60|61': {'00': ['10'], '11': ['10']}, '61|63': {'00': ['10'], '11': ['10']}, '56|53': {'00': ['10']}, '57|54': {'00': ['10']}, '58|56': {'00': ['10']}, '59|58': {'00': ['10']}, '60|60': {'00': ['10']}, '61|62': {'00': ['10']}, '48|43': {'01': ['10'], '11': ['10', '00']}, '46|40': {'01': ['10'], '11': ['00']}, '47|41': {'01': ['10'], '11': ['00', '10']}, '49|44': {'01': ['10'], '11': ['10', '00']}, '50|45': {'01': ['10'], '11': ['00', '10']}, '51|46': {'01': ['10'], '11': ['10', '00']}, '52|48': {'01': ['10'], '11': ['10', '00']}, '48|42': {'01': ['10'], '11': ['10', '00']}, '45|38': {'01': ['10'], '11': ['00']}, '45|39': {'11': ['00']}, '46|39': {'01': ['10'], '11': ['00']}, '47|40': {'01': ['10'], '11': ['00', '10']}, '49|43': {'01': ['10'], '11': ['10', '00']}, '48|41': {'01': ['10'], '11': ['10', '00']}, '50|44': {'01': ['10'], '11': ['10', '00']}, '52|47': {'01': ['10'], '11': ['10', '00']}, '51|45': {'01': ['10'], '11': ['10', '00']}, '44|37': {'01': ['00', '11']}, '45|37': {'01': ['00', '10']}, '46|38': {'01': ['10'], '11': ['00', '10']}, '47|39': {'01': ['10'], '11': ['00', '10']}, '49|42': {'01': ['10'], '11': ['10', '00']}, '48|40': {'01': ['10'], '11': ['10']}, '50|43': {'01': ['10'], '11': ['10']}, '52|46': {'01': ['10'], '11': ['10']}, '53|49': {'01': ['10'], '11': ['10']}, '53|48': {'01': ['10'], '11': ['10']}, '54|50': {'00': ['10']}, '46|37': {'01': ['10']}, '47|38': {'01': ['10']}, '49|41': {'01': ['10']}, '51|44': {'01': ['10']}, '53|47': {'01': ['10']}, '48|39': {'01': ['10']}, '50|42': {'01': ['10']}, '52|45': {'01': ['10']}}
#visitedB = {'43|37': {'10': ['00']}, '44|37': {'01': ['00', '10']}, '44|38': {'11': ['11']}, '44|39': {'10': ['00']}, '45|39': {'01': ['10']}, '45|40': {'11': ['00']}, '46|40': {'01': ['10']}, '46|41': {'11': ['00']}, '47|41': {'01': ['10'], '11': ['00']}, '47|42': {'11': ['00', '10']}, '47|43': {'11': ['00']}, '48|43': {'01': ['10'], '11': ['10', '00']}, '48|44': {'11': ['00', '10']}, '48|45': {'11': ['00']}, '49|45': {'01': ['10'], '11': ['00', '10']}, '49|46': {'11': ['00', '10']}, '49|47': {'11': ['00']}, '50|47': {'01': ['10'], '11': ['00', '10']}, '50|48': {'11': ['00', '10']}, '50|49': {'11': ['00', '10']}, '50|50': {'11': []}, '51|49': {'01': ['10'], '11': ['10', '00']}, '51|50': {'11': ['01']}, '52|50': {'00': ['10'], '11': ['01', '10']}, '52|51': {'11': []}, '51|48': {'01': ['10'], '11': ['10', '00']}, '50|46': {'01': ['10'], '11': ['00', '10']}, '51|47': {'01': ['10'], '11': ['10', '00']}, '52|49': {'01': ['10'], '11': ['10', '00']}, '53|50': {'00': ['10'], '11': ['10', '01']}, '53|51': {'11': ['01']}, '54|51': {'00': ['10'], '11': ['10']}, '54|52': {'11': ['01']}, '55|52': {'00': ['10']}, '55|53': {'11': ['01', '10']}, '55|54': {'11': ['01']}, '56|54': {'00': ['10'], '11': ['01', '10']}, '56|55': {'11': ['01']}, '57|55': {'00': ['10'], '11': ['10']}, '57|56': {'11': ['10', '01']}, '57|57': {'11': ['01', '10']}, '57|58': {'11': ['01']}, '58|58': {'00': ['10'], '11': ['10', '01']}, '58|59': {'11': ['10', '01']}, '58|60': {'11': ['01', '10']}, '58|61': {'11': ['01']}, '59|61': {'00': ['10'], '11': ['10', '01']}, '59|62': {'11': ['10', '01']}, '59|63': {'11': ['01', '10']}, '59|64': {'11': ['01', '10']}, '59|65': {'11': ['01', '10']}, '59|66': {'11': ['01', '10']}, '59|67': {'11': ['01', '10']}, '59|68': {'11': ['01', '10']}, '59|69': {'11': ['01', '10']}, '59|70': {'11': ['01', '10']}, '59|71': {'11': ['01', '10']}, '59|72': {'11': ['01', '10']}, '59|73': {'11': ['01', '10']}, '59|74': {'11': ['01', '10']}, '59|75': {'11': ['01', '10']}, '59|76': {'11': ['01', '10']}, '59|77': {'11': ['01', '10']}, '59|78': {'11': ['01', '10']}, '59|79': {'11': ['01', '10']}, '59|80': {'11': ['01', '10']}, '59|81': {'11': ['01', '10']}, '59|82': {'11': ['01', '10']}, '59|83': {'11': ['01', '10']}, '59|84': {'11': ['01', '10']}, '59|85': {'11': ['01', '10']}, '59|86': {'11': ['01', '10']}, '59|87': {'11': ['01', '10']}, '59|88': {'11': ['01', '10']}, '59|89': {'11': ['01', '10']}, '59|90': {'11': ['01', '10']}, '59|91': {'11': ['01', '10']}, '59|92': {'11': ['01', '10']}, '59|93': {'11': ['01', '10']}, '59|94': {'11': ['01', '10']}, '59|95': {'11': ['01', '10']}, '59|96': {'11': ['01', '10']}, '59|97': {'11': ['01', '10']}, '59|98': {'11': ['01', '10']}, '59|99': {'11': ['01', '10']}, '60|99': {'00': ['10'], '11': ['10', '01']}, '60|98': {'00': ['10'], '11': ['10', '01']}, '60|97': {'00': ['10'], '11': ['10', '01']}, '61|99': {'00': ['10'], '11': ['10']}, '60|96': {'00': ['10'], '11': ['10', '01']}, '61|98': {'00': ['10'], '11': ['10']}, '60|95': {'00': ['10'], '11': ['10', '01']}, '61|97': {'00': ['10'], '11': ['10']}, '60|94': {'00': ['10'], '11': ['10', '01']}, '61|96': {'00': ['10'], '11': ['10']}, '60|93': {'00': ['10'], '11': ['10', '01']}, '61|95': {'00': ['10'], '11': ['10', '01']}, '60|92': {'00': ['10'], '11': ['10', '01']}, '61|94': {'00': ['10'], '11': ['10', '01']}, '60|91': {'00': ['10'], '11': ['10', '01']}, '61|93': {'00': ['10'], '11': ['10', '01']}, '62|95': {'00': ['10'], '11': ['10']}, '62|96': {'11': ['10']}, '62|97': {'11': ['10']}, '62|98': {'11': ['10']}, '62|99': {'11': ['10']}, '60|90': {'00': ['10'], '11': ['10', '01']}, '61|92': {'00': ['10'], '11': ['10', '01']}, '62|94': {'00': ['10'], '11': ['10']}, '60|89': {'00': ['10'], '11': ['10', '01']}, '61|91': {'00': ['10'], '11': ['10', '01']}, '62|93': {'00': ['10'], '11': ['10']}, '60|88': {'00': ['10'], '11': ['10', '01']}, '61|90': {'00': ['10'], '11': ['10', '01']}, '62|92': {'00': ['10'], '11': ['10']}, '60|87': {'00': ['10'], '11': ['10', '01']}, '61|89': {'00': ['10'], '11': ['10', '01']}, '62|91': {'00': ['10'], '11': ['10']}, '60|86': {'00': ['10'], '11': ['10', '01']}, '61|88': {'00': ['10'], '11': ['10', '01']}, '62|90': {'00': ['10'], '11': ['10']}, '60|85': {'00': ['10'], '11': ['10', '01']}, '61|87': {'00': ['10'], '11': ['10', '01']}, '62|89': {'00': ['10'], '11': ['10']}, '60|84': {'00': ['10'], '11': ['10', '01']}, '61|86': {'00': ['10'], '11': ['10', '01']}, '62|88': {'00': ['10'], '11': ['10']}, '60|83': {'00': ['10'], '11': ['10', '01']}, '61|85': {'00': ['10'], '11': ['10', '01']}, '62|87': {'00': ['10'], '11': ['10']}, '60|82': {'00': ['10'], '11': ['10', '01']}, '61|84': {'00': ['10'], '11': ['10', '01']}, '62|86': {'00': ['10'], '11': ['10']}, '60|81': {'00': ['10'], '11': ['10', '01']}, '61|83': {'00': ['10'], '11': ['10', '01']}, '62|85': {'00': ['10'], '11': ['10']}, '60|80': {'00': ['10'], '11': ['10', '01']}, '61|82': {'00': ['10'], '11': ['10', '01']}, '62|84': {'00': ['10'], '11': ['10']}, '60|79': {'00': ['10'], '11': ['10', '01']}, '61|81': {'00': ['10'], '11': ['10', '01']}, '62|83': {'00': ['10'], '11': ['10']}, '60|78': {'00': ['10'], '11': ['10', '01']}, '61|80': {'00': ['10'], '11': ['10', '01']}, '62|82': {'00': ['10'], '11': ['10']}, '60|77': {'00': ['10'], '11': ['10', '01']}, '61|79': {'00': ['10'], '11': ['10', '01']}, '62|81': {'00': ['10'], '11': ['10']}, '60|76': {'00': ['10'], '11': ['10', '01']}, '61|78': {'00': ['10'], '11': ['10', '01']}, '62|80': {'00': ['10'], '11': ['10']}, '60|75': {'00': ['10'], '11': ['10', '01']}, '61|77': {'00': ['10'], '11': ['10', '01']}, '62|79': {'00': ['10'], '11': ['10']}, '60|74': {'00': ['10'], '11': ['10', '01']}, '61|76': {'00': ['10'], '11': ['10', '01']}, '62|78': {'00': ['10'], '11': ['10']}, '60|73': {'00': ['10'], '11': ['10', '01']}, '61|75': {'00': ['10'], '11': ['10', '01']}, '62|77': {'00': ['10'], '11': ['10']}, '60|72': {'00': ['10'], '11': ['10', '01']}, '61|74': {'00': ['10'], '11': ['10', '01']}, '62|76': {'00': ['10'], '11': ['10']}, '60|71': {'00': ['10'], '11': ['10', '01']}, '61|73': {'00': ['10'], '11': ['10', '01']}, '62|75': {'00': ['10'], '11': ['10']}, '60|70': {'00': ['10'], '11': ['10', '01']}, '61|72': {'00': ['10'], '11': ['10', '01']}, '62|74': {'00': ['10'], '11': ['10']}, '60|69': {'00': ['10'], '11': ['10', '01']}, '61|71': {'00': ['10'], '11': ['10']}, '62|73': {'00': ['10'], '11': ['10']}, '60|68': {'00': ['10'], '11': ['10', '01']}, '61|70': {'00': ['10'], '11': ['10']}, '62|72': {'00': ['10']}, '60|67': {'00': ['10'], '11': ['10', '01']}, '61|69': {'00': ['10'], '11': ['10']}, '60|66': {'00': ['10'], '11': ['10', '01']}, '61|68': {'00': ['10'], '11': ['10']}, '60|65': {'00': ['10'], '11': ['10', '01']}, '61|67': {'00': ['10'], '11': ['10']}, '60|64': {'00': ['10'], '11': ['10', '01']}, '61|66': {'00': ['10'], '11': ['10']}, '60|63': {'00': ['10'], '11': ['10', '01']}, '61|65': {'00': ['10'], '11': ['10']}, '59|60': {'00': ['10'], '11': ['10', '01']}, '60|62': {'00': ['10'], '11': ['10', '01']}, '61|64': {'00': ['10'], '11': ['10']}, '58|57': {'00': ['10'], '11': ['10']}, '59|59': {'00': ['10'], '11': ['10']}, '60|61': {'00': ['10'], '11': ['10']}, '61|63': {'00': ['10'], '11': ['10']}, '56|53': {'00': ['10']}, '57|54': {'00': ['10']}, '58|56': {'00': ['10']}, '59|58': {'00': ['10']}, '60|60': {'00': ['10']}, '61|62': {'00': ['10']}, '49|44': {'01': ['10'], '11': ['00', '10']}, '50|45': {'01': ['10'], '11': ['00', '10']}, '51|46': {'01': ['10'], '11': ['10', '00']}, '52|48': {'01': ['10'], '11': ['10', '00']}, '48|42': {'01': ['10'], '11': ['00', '10']}, '45|37': {'01': ['10']}, '45|38': {'11': ['00']}, '46|38': {'01': ['10']}, '46|39': {'11': ['00']}, '47|39': {'01': ['10']}, '47|40': {'11': ['00', '10']}, '48|41': {'01': ['10'], '11': ['10']}, '49|43': {'01': ['10'], '11': ['10', '00']}, '50|44': {'01': ['10'], '11': ['10']}, '52|47': {'01': ['10'], '11': ['10']}, '51|45': {'01': ['10']}, '49|42': {'01': ['10']}, '48|40': {'01': ['10']}, '50|43': {'01': ['10']}, '52|46': {'01': ['10']}, '53|49': {'01': ['10'], '11': ['10']}, '53|48': {'01': ['10']}, '54|50': {'00': ['10']}}

#from 43,37 B 00
#visitedB = {'43|37': {'00': ['01', '10']}, '43|38': {'11': ['01']}, '44|38': {'00': ['01', '10'], '11': ['01', '11']}, '44|39': {'11': ['01'], '10': ['01']}, '45|39': {'00': ['11', '01'], '10': ['01']}, '45|40': {'10': ['01']}, '46|40': {'00': ['11'], '10': ['01', '11']}, '46|41': {'10': ['01']}, '47|41': {'00': ['11'], '10': ['01', '11']}, '47|42': {'10': ['01', '11']}, '47|43': {'10': ['01']}, '48|43': {'00': ['11'], '10': ['01', '11']}, '48|44': {'10': ['01', '11']}, '48|45': {'10': ['01', '11']}, '48|46': {'10': ['01']}, '49|46': {'00': ['11'], '10': ['01', '11']}, '49|47': {'10': ['01', '11']}, '49|48': {'10': ['01', '11']}, '49|49': {'10': ['01']}, '50|49': {'00': ['11'], '10': ['11', '01']}, '50|50': {'10': []}, '50|48': {'00': ['11'], '10': ['11', '01']}, '50|47': {'00': ['11'], '10': ['11', '01']}, '49|45': {'00': ['11'], '10': ['11', '01']}, '51|49': {'00': ['11'], '10': ['11', '01']}, '51|50': {'10': ['00']}, '52|50': {'01': ['11'], '10': ['00', '11']}, '52|51': {'10': ['00']}, '53|51': {'01': ['00'], '10': ['00']}, '54|51': {'01': ['11'], '10': ['11']}, '54|52': {'10': ['00']}, '55|52': {'01': ['11']}, '55|53': {'10': ['00', '11']}, '55|54': {'10': ['00']}, '56|54': {'01': ['11'], '10': ['00', '11']}, '56|55': {'10': ['00']}, '57|55': {'01': ['11'], '10': ['11']}, '57|56': {'10': ['11', '00']}, '57|57': {'10': ['00']}, '58|57': {'01': ['11'], '10': ['11']}, '58|58': {'10': ['11', '00']}, '58|59': {'10': ['00', '11']}, '58|60': {'10': ['00', '11']}, '58|61': {'10': ['00', '11']}, '58|62': {'10': ['00', '11']}, '58|63': {'10': ['00', '11']}, '58|64': {'10': ['00', '11']}, '58|65': {'10': ['00']}, '59|65': {'01': ['11'], '10': ['11', '00']}, '59|66': {'10': ['11', '00']}, '59|67': {'10': ['00', '11']}, '59|68': {'10': ['00', '11']}, '59|69': {'10': ['00', '11']}, '59|70': {'10': ['00', '11']}, '59|71': {'10': ['00', '11']}, '59|72': {'10': ['00', '11']}, '59|73': {'10': ['00', '11']}, '59|74': {'10': ['00', '11']}, '59|75': {'10': ['00', '11']}, '59|76': {'10': ['00', '11']}, '59|77': {'10': ['00', '11']}, '59|78': {'10': ['00', '11']}, '59|79': {'10': ['00', '11']}, '59|80': {'10': ['00', '11']}, '59|81': {'10': ['00', '11']}, '59|82': {'10': ['00', '11']}, '59|83': {'10': ['00', '11']}, '59|84': {'10': ['00', '11']}, '59|85': {'10': ['00', '11']}, '59|86': {'10': ['00', '11']}, '59|87': {'10': ['00', '11']}, '59|88': {'10': ['00', '11']}, '59|89': {'10': ['00', '11']}, '59|90': {'10': ['00', '11']}, '59|91': {'10': ['00', '11']}, '59|92': {'10': ['00', '11']}, '59|93': {'10': ['00', '11']}, '59|94': {'10': ['00', '11']}, '59|95': {'10': ['00', '11']}, '59|96': {'10': ['00', '11']}, '59|97': {'10': ['00', '11']}, '59|98': {'10': ['00', '11']}, '59|99': {'10': ['00', '11']}, '60|99': {'01': ['11'], '10': ['11', '00']}, '60|98': {'01': ['11'], '10': ['11', '00']}, '60|97': {'01': ['11'], '10': ['11', '00']}, '60|96': {'01': ['11'], '10': ['11', '00']}, '61|99': {'01': ['11'], '10': ['11', '00']}, '60|95': {'01': ['11'], '10': ['11', '00']}, '61|98': {'01': ['11'], '10': ['11', '00']}, '60|94': {'01': ['11'], '10': ['11', '00']}, '61|97': {'01': ['11'], '10': ['11', '00']}, '60|93': {'01': ['11'], '10': ['11', '00']}, '61|96': {'01': ['11'], '10': ['11', '00']}, '62|99': {'01': ['11'], '10': ['11', '00']}, '60|92': {'01': ['11'], '10': ['11', '00']}, '61|95': {'01': ['11'], '10': ['11', '00']}, '62|98': {'01': ['11'], '10': ['11', '00']}, '60|91': {'01': ['11'], '10': ['11', '00']}, '61|94': {'01': ['11'], '10': ['11', '00']}, '62|97': {'01': ['11'], '10': ['11', '00']}, '60|90': {'01': ['11'], '10': ['11', '00']}, '61|93': {'01': ['11'], '10': ['11', '00']}, '62|96': {'01': ['11'], '10': ['11', '00']}, '63|99': {'01': ['11'], '10': ['11', '00']}, '60|89': {'01': ['11'], '10': ['11', '00']}, '61|92': {'01': ['11'], '10': ['11', '00']}, '62|95': {'01': ['11'], '10': ['11', '00']}, '63|98': {'01': ['11'], '10': ['11', '00']}, '60|88': {'01': ['11'], '10': ['11', '00']}, '61|91': {'01': ['11'], '10': ['11', '00']}, '62|94': {'01': ['11'], '10': ['11', '00']}, '63|97': {'01': ['11'], '10': ['11', '00']}, '60|87': {'01': ['11'], '10': ['11', '00']}, '61|90': {'01': ['11'], '10': ['11', '00']}, '62|93': {'01': ['11'], '10': ['11', '00']}, '63|96': {'01': ['11'], '10': ['11', '00']}, '64|99': {'01': ['11'], '10': ['11', '00']}, '60|86': {'01': ['11'], '10': ['11', '00']}, '61|89': {'01': ['11'], '10': ['11', '00']}, '62|92': {'01': ['11'], '10': ['11', '00']}, '63|95': {'01': ['11'], '10': ['11', '00']}, '64|98': {'01': ['11'], '10': ['11', '00']}, '60|85': {'01': ['11'], '10': ['11', '00']}, '61|88': {'01': ['11'], '10': ['11', '00']}, '62|91': {'01': ['11'], '10': ['11', '00']}, '63|94': {'01': ['11'], '10': ['11', '00']}, '64|97': {'01': ['11'], '10': ['11', '00']}, '60|84': {'01': ['11'], '10': ['11', '00']}, '61|87': {'01': ['11'], '10': ['11', '00']}, '62|90': {'01': ['11'], '10': ['11', '00']}, '63|93': {'01': ['11'], '10': ['11', '00']}, '64|96': {'01': ['11'], '10': ['11', '00']}, '65|99': {'01': ['11'], '10': ['11', '00']}, '60|83': {'01': ['11'], '10': ['11', '00']}, '61|86': {'01': ['11'], '10': ['11', '00']}, '62|89': {'01': ['11'], '10': ['11', '00']}, '63|92': {'01': ['11'], '10': ['11', '00']}, '64|95': {'01': ['11'], '10': ['11', '00']}, '65|98': {'01': ['11'], '10': ['11', '00']}, '60|82': {'01': ['11'], '10': ['11', '00']}, '61|85': {'01': ['11'], '10': ['11', '00']}, '62|88': {'01': ['11'], '10': ['11', '00']}, '63|91': {'01': ['11'], '10': ['11', '00']}, '64|94': {'01': ['11'], '10': ['11', '00']}, '65|97': {'01': ['11'], '10': ['11', '00']}, '60|81': {'01': ['11'], '10': ['11', '00']}, '61|84': {'01': ['11'], '10': ['11', '00']}, '62|87': {'01': ['11'], '10': ['11', '00']}, '63|90': {'01': ['11'], '10': ['11', '00']}, '64|93': {'01': ['11'], '10': ['11', '00']}, '65|96': {'01': ['11'], '10': ['11', '00']}, '66|99': {'01': ['11'], '10': ['11', '00']}, '60|80': {'01': ['11'], '10': ['11', '00']}, '61|83': {'01': ['11'], '10': ['11', '00']}, '62|86': {'01': ['11'], '10': ['11', '00']}, '63|89': {'01': ['11'], '10': ['11', '00']}, '64|92': {'01': ['11'], '10': ['11', '00']}, '65|95': {'01': ['11'], '10': ['11', '00']}, '66|98': {'01': ['11'], '10': ['11', '00']}, '60|79': {'01': ['11'], '10': ['11', '00']}, '61|82': {'01': ['11'], '10': ['11', '00']}, '62|85': {'01': ['11'], '10': ['11', '00']}, '63|88': {'01': ['11'], '10': ['11', '00']}, '64|91': {'01': ['11'], '10': ['11', '00']}, '65|94': {'01': ['11'], '10': ['11', '00']}, '66|97': {'01': ['11'], '10': ['11', '00']}, '60|78': {'01': ['11'], '10': ['11', '00']}, '61|81': {'01': ['11'], '10': ['11', '00']}, '62|84': {'01': ['11'], '10': ['11', '00']}, '63|87': {'01': ['11'], '10': ['11', '00']}, '64|90': {'01': ['11'], '10': ['11', '00']}, '65|93': {'01': ['11'], '10': ['11', '00']}, '66|96': {'01': ['11'], '10': ['11', '00']}, '67|99': {'01': ['11'], '10': ['11', '00']}, '60|77': {'01': ['11'], '10': ['11', '00']}, '61|80': {'01': ['11'], '10': ['11', '00']}, '62|83': {'01': ['11'], '10': ['11', '00']}, '63|86': {'01': ['11'], '10': ['11', '00']}, '64|89': {'01': ['11'], '10': ['11', '00']}, '65|92': {'01': ['11'], '10': ['11', '00']}, '66|95': {'01': ['11'], '10': ['11', '00']}, '67|98': {'01': ['11'], '10': ['11', '00']}, '60|76': {'01': ['11'], '10': ['11', '00']}, '61|79': {'01': ['11'], '10': ['11', '00']}, '62|82': {'01': ['11'], '10': ['11', '00']}, '63|85': {'01': ['11'], '10': ['11', '00']}, '64|88': {'01': ['11'], '10': ['11', '00']}, '65|91': {'01': ['11'], '10': ['11', '00']}, '66|94': {'01': ['11'], '10': ['11', '00']}, '67|97': {'01': ['11'], '10': ['11', '00']}, '60|75': {'01': ['11'], '10': ['11', '00']}, '61|78': {'01': ['11'], '10': ['11', '00']}, '62|81': {'01': ['11'], '10': ['11', '00']}, '63|84': {'01': ['11'], '10': ['11', '00']}, '64|87': {'01': ['11'], '10': ['11', '00']}, '65|90': {'01': ['11'], '10': ['11', '00']}, '66|93': {'01': ['11'], '10': ['11', '00']}, '67|96': {'01': ['11'], '10': ['11', '00']}, '68|99': {'01': ['11'], '10': ['11', '00']}, '60|74': {'01': ['11'], '10': ['11', '00']}, '61|77': {'01': ['11'], '10': ['11', '00']}, '62|80': {'01': ['11'], '10': ['11', '00']}, '63|83': {'01': ['11'], '10': ['11', '00']}, '64|86': {'01': ['11'], '10': ['11', '00']}, '65|89': {'01': ['11'], '10': ['11', '00']}, '66|92': {'01': ['11'], '10': ['11', '00']}, '67|95': {'01': ['11'], '10': ['11', '00']}, '68|98': {'01': ['11'], '10': ['11', '00']}, '60|73': {'01': ['11'], '10': ['11', '00']}, '61|76': {'01': ['11'], '10': ['11', '00']}, '62|79': {'01': ['11'], '10': ['11', '00']}, '63|82': {'01': ['11'], '10': ['11', '00']}, '64|85': {'01': ['11'], '10': ['11', '00']}, '65|88': {'01': ['11'], '10': ['11', '00']}, '66|91': {'01': ['11'], '10': ['11', '00']}, '67|94': {'01': ['11'], '10': ['11', '00']}, '68|97': {'01': ['11'], '10': ['11', '00']}, '60|72': {'01': ['11'], '10': ['11', '00']}, '61|75': {'01': ['11'], '10': ['11', '00']}, '62|78': {'01': ['11'], '10': ['11', '00']}, '63|81': {'01': ['11'], '10': ['11', '00']}, '64|84': {'01': ['11'], '10': ['11', '00']}, '65|87': {'01': ['11'], '10': ['11', '00']}, '66|90': {'01': ['11'], '10': ['11', '00']}, '67|93': {'01': ['11'], '10': ['11', '00']}, '68|96': {'01': ['11'], '10': ['11', '00']}, '69|99': {'01': ['11'], '10': ['11', '00']}, '60|71': {'01': ['11'], '10': ['11', '00']}, '61|74': {'01': ['11'], '10': ['11', '00']}, '62|77': {'01': ['11'], '10': ['11', '00']}, '63|80': {'01': ['11'], '10': ['11', '00']}, '64|83': {'01': ['11'], '10': ['11', '00']}, '65|86': {'01': ['11'], '10': ['11', '00']}, '66|89': {'01': ['11'], '10': ['11', '00']}, '67|92': {'01': ['11'], '10': ['11', '00']}, '68|95': {'01': ['11'], '10': ['11', '00']}, '69|98': {'01': ['11'], '10': ['11', '00']}, '60|70': {'01': ['11'], '10': ['11', '00']}, '61|73': {'01': ['11'], '10': ['11', '00']}, '62|76': {'01': ['11'], '10': ['11', '00']}, '63|79': {'01': ['11'], '10': ['11', '00']}, '64|82': {'01': ['11'], '10': ['11', '00']}, '65|85': {'01': ['11'], '10': ['11', '00']}, '66|88': {'01': ['11'], '10': ['11', '00']}, '67|91': {'01': ['11'], '10': ['11', '00']}, '68|94': {'01': ['11'], '10': ['11', '00']}, '69|97': {'01': ['11'], '10': ['11', '00']}, '60|69': {'01': ['11'], '10': ['11', '00']}, '61|72': {'01': ['11'], '10': ['11', '00']}, '62|75': {'01': ['11'], '10': ['11', '00']}, '63|78': {'01': ['11'], '10': ['11', '00']}, '64|81': {'01': ['11'], '10': ['11', '00']}, '65|84': {'01': ['11'], '10': ['11', '00']}, '66|87': {'01': ['11'], '10': ['11', '00']}, '67|90': {'01': ['11'], '10': ['11', '00']}, '68|93': {'01': ['11'], '10': ['11', '00']}, '69|96': {'01': ['11'], '10': ['11', '00']}, '70|99': {'01': ['11'], '10': ['11', '00']}, '60|68': {'01': ['11'], '10': ['11', '00']}, '61|71': {'01': ['11'], '10': ['11', '00']}, '62|74': {'01': ['11'], '10': ['11', '00']}, '63|77': {'01': ['11'], '10': ['11', '00']}, '64|80': {'01': ['11'], '10': ['11', '00']}, '65|83': {'01': ['11'], '10': ['11', '00']}, '66|86': {'01': ['11'], '10': ['11', '00']}, '67|89': {'01': ['11'], '10': ['11', '00']}, '68|92': {'01': ['11'], '10': ['11', '00']}, '69|95': {'01': ['11'], '10': ['11', '00']}, '70|98': {'01': ['11'], '10': ['11', '00']}, '60|67': {'01': ['11'], '10': ['11', '00']}, '61|70': {'01': ['11'], '10': ['11', '00']}, '62|73': {'01': ['11'], '10': ['11', '00']}, '63|76': {'01': ['11'], '10': ['11', '00']}, '64|79': {'01': ['11'], '10': ['11', '00']}, '65|82': {'01': ['11'], '10': ['11', '00']}, '66|85': {'01': ['11'], '10': ['11', '00']}, '67|88': {'01': ['11'], '10': ['11', '00']}, '68|91': {'01': ['11'], '10': ['11', '00']}, '69|94': {'01': ['11'], '10': ['11', '00']}, '70|97': {'01': ['11'], '10': ['11', '00']}, '59|64': {'01': ['11'], '10': ['11', '00']}, '60|66': {'01': ['11'], '10': ['11', '00']}, '61|69': {'01': ['11'], '10': ['11', '00']}, '62|72': {'01': ['11'], '10': ['11', '00']}, '63|75': {'01': ['11'], '10': ['11', '00']}, '64|78': {'01': ['11'], '10': ['11', '00']}, '65|81': {'01': ['11'], '10': ['11', '00']}, '66|84': {'01': ['11'], '10': ['11', '00']}, '67|87': {'01': ['11'], '10': ['11', '00']}, '68|90': {'01': ['11'], '10': ['11', '00']}, '69|93': {'01': ['11'], '10': ['11', '00']}, '70|96': {'01': ['11'], '10': ['11', '00']}, '71|99': {'01': ['11'], '10': ['11', '00']}, '59|63': {'01': ['11'], '10': ['11', '00']}, '60|65': {'01': ['11'], '10': ['11', '00']}, '61|68': {'01': ['11'], '10': ['11', '00']}, '62|71': {'01': ['11'], '10': ['11', '00']}, '63|74': {'01': ['11'], '10': ['11', '00']}, '64|77': {'01': ['11'], '10': ['11', '00']}, '65|80': {'01': ['11'], '10': ['11', '00']}, '66|83': {'01': ['11'], '10': ['11', '00']}, '67|86': {'01': ['11'], '10': ['11', '00']}, '68|89': {'01': ['11'], '10': ['11', '00']}, '69|92': {'01': ['11'], '10': ['11', '00']}, '70|95': {'01': ['11'], '10': ['11', '00']}, '71|98': {'01': ['11'], '10': ['11', '00']}, '59|62': {'01': ['11'], '10': ['11', '00']}, '60|64': {'01': ['11'], '10': ['11', '00']}, '61|67': {'01': ['11'], '10': ['11', '00']}, '62|70': {'01': ['11'], '10': ['11', '00']}, '63|73': {'01': ['11'], '10': ['11', '00']}, '64|76': {'01': ['11'], '10': ['11', '00']}, '65|79': {'01': ['11'], '10': ['11', '00']}, '66|82': {'01': ['11'], '10': ['11', '00']}, '67|85': {'01': ['11'], '10': ['11', '00']}, '68|88': {'01': ['11'], '10': ['11', '00']}, '69|91': {'01': ['11'], '10': ['11', '00']}, '70|94': {'01': ['11'], '10': ['11', '00']}, '71|97': {'01': ['11'], '10': ['11', '00']}, '59|61': {'01': ['11'], '10': ['11', '00']}, '60|63': {'01': ['11'], '10': ['11']}, '61|66': {'01': ['11'], '10': ['11']}, '62|69': {'01': ['11'], '10': ['11']}, '63|72': {'01': ['11'], '10': ['11']}, '64|75': {'01': ['11'], '10': ['11']}, '65|78': {'01': ['11'], '10': ['11']}, '66|81': {'01': ['11'], '10': ['11']}, '67|84': {'01': ['11'], '10': ['11']}, '68|87': {'01': ['11'], '10': ['11']}, '69|90': {'01': ['11'], '10': ['11']}, '70|93': {'01': ['11'], '10': ['11']}, '71|96': {'01': ['11'], '10': ['11']}, '72|99': {'01': ['11'], '10': ['11']}, '59|60': {'01': ['11'], '10': ['11']}, '59|59': {'01': ['11'], '10': ['11']}, '60|62': {'01': ['11'], '10': ['11']}, '61|65': {'01': ['11'], '10': ['11']}, '62|68': {'01': ['11'], '10': ['11']}, '63|71': {'01': ['11'], '10': ['11']}, '64|74': {'01': ['11'], '10': ['11']}, '65|77': {'01': ['11'], '10': ['11']}, '66|80': {'01': ['11'], '10': ['11']}, '67|83': {'01': ['11'], '10': ['11']}, '68|86': {'01': ['11'], '10': ['11']}, '69|89': {'01': ['11'], '10': ['11']}, '70|92': {'01': ['11'], '10': ['11']}, '71|95': {'01': ['11'], '10': ['11']}, '72|98': {'01': ['11'], '10': ['11']}, '56|53': {'01': ['11']}, '58|56': {'01': ['11']}, '59|58': {'01': ['11']}, '60|61': {'01': ['11']}, '61|64': {'01': ['11']}, '62|67': {'01': ['11']}, '63|70': {'01': ['11']}, '64|73': {'01': ['11']}, '65|76': {'01': ['11']}, '66|79': {'01': ['11']}, '67|82': {'01': ['11']}, '68|85': {'01': ['11']}, '69|88': {'01': ['11']}, '70|91': {'01': ['11']}, '71|94': {'01': ['11']}, '72|97': {'01': ['11']}, '57|54': {'01': ['11']}, '50|46': {'00': ['11'], '10': ['11', '01']}, '49|44': {'00': ['11'], '10': ['11', '01']}, '51|48': {'00': ['11'], '10': ['11', '01']}, '48|42': {'00': ['11'], '10': ['01', '11']}, '50|45': {'00': ['11'], '10': ['11', '01']}, '49|43': {'00': ['11'], '10': ['11', '01']}, '51|47': {'00': ['11'], '10': ['11', '01']}, '52|49': {'00': ['11'], '10': ['11']}, '53|50': {'01': ['11'], '10': ['00']}, '45|38': {'00': ['01', '11']}, '46|39': {'00': ['11'], '10': ['01', '11']}, '47|40': {'00': ['11'], '10': ['01', '11']}, '48|41': {'00': ['11'], '10': ['01', '11']}, '50|44': {'00': ['11'], '10': ['11', '01']}, '49|42': {'00': ['11'], '10': ['11', '01']}, '51|46': {'00': ['11'], '10': ['11', '01']}, '52|48': {'00': ['11'], '10': ['11', '01']}, '46|38': {'00': ['11'], '10': ['01', '11']}, '47|39': {'00': ['11'], '10': ['01', '11']}, '48|40': {'00': ['11'], '10': ['01', '11']}, '50|43': {'00': ['11'], '10': ['11']}, '49|41': {'00': ['11'], '10': ['11']}, '51|45': {'00': ['11'], '10': ['11']}, '52|47': {'00': ['11'], '10': ['11']}, '44|37': {'00': ['01', '10']}, '45|37': {'00': ['01']}, '46|37': {'00': ['11']}, '47|38': {'00': ['11']}, '48|39': {'00': ['11']}, '50|42': {'00': ['11']}, '49|40': {'00': ['11']}, '51|44': {'00': ['11']}, '52|46': {'00': ['11']}, '53|48': {'00': ['11']}, '53|49': {'10': ['11']}, '54|50': {'01': ['11']}}
#from 43,37 F 10
#visitedF = {'43|37': {'10': ['00']}, '44|37': {'01': ['00', '10']}, '44|38': {'11': ['11']}, '44|39': {'10': ['00']}, '45|39': {'01': ['10']}, '45|40': {'11': ['00']}, '46|40': {'01': ['10']}, '46|41': {'11': ['00']}, '47|41': {'01': ['10'], '11': ['00']}, '47|42': {'11': ['00', '10']}, '47|43': {'11': ['00']}, '48|43': {'01': ['10'], '11': ['10', '00']}, '48|44': {'11': ['00', '10']}, '48|45': {'11': ['00']}, '49|45': {'01': ['10'], '11': ['00', '10']}, '49|46': {'11': ['00', '10']}, '49|47': {'11': ['00']}, '50|47': {'01': ['10'], '11': ['00', '10']}, '50|48': {'11': ['00', '10']}, '50|49': {'11': ['00', '10']}, '50|50': {'11': []}, '51|49': {'01': ['10'], '11': ['10', '00']}, '51|50': {'11': ['01']}, '52|50': {'00': ['10'], '11': ['01', '10']}, '52|51': {'11': []}, '51|48': {'01': ['10'], '11': ['10', '00']}, '50|46': {'01': ['10'], '11': ['00', '10']}, '51|47': {'01': ['10'], '11': ['10', '00']}, '52|49': {'01': ['10'], '11': ['10', '00']}, '53|50': {'00': ['10'], '11': ['10', '01']}, '53|51': {'11': ['01']}, '54|51': {'00': ['10'], '11': ['10']}, '54|52': {'11': ['01']}, '55|52': {'00': ['10']}, '55|53': {'11': ['01', '10']}, '55|54': {'11': ['01']}, '56|54': {'00': ['10'], '11': ['01', '10']}, '56|55': {'11': ['01']}, '57|55': {'00': ['10'], '11': ['10']}, '57|56': {'11': ['10', '01']}, '57|57': {'11': ['01', '10']}, '57|58': {'11': ['01']}, '58|58': {'00': ['10'], '11': ['10', '01']}, '58|59': {'11': ['10', '01']}, '58|60': {'11': ['01', '10']}, '58|61': {'11': ['01']}, '59|61': {'00': ['10'], '11': ['10', '01']}, '59|62': {'11': ['10', '01']}, '59|63': {'11': ['01', '10']}, '59|64': {'11': ['01', '10']}, '59|65': {'11': ['01', '10']}, '59|66': {'11': ['01', '10']}, '59|67': {'11': ['01', '10']}, '59|68': {'11': ['01', '10']}, '59|69': {'11': ['01', '10']}, '59|70': {'11': ['01', '10']}, '59|71': {'11': ['01', '10']}, '59|72': {'11': ['01', '10']}, '59|73': {'11': ['01', '10']}, '59|74': {'11': ['01', '10']}, '59|75': {'11': ['01', '10']}, '59|76': {'11': ['01', '10']}, '59|77': {'11': ['01', '10']}, '59|78': {'11': ['01', '10']}, '59|79': {'11': ['01', '10']}, '59|80': {'11': ['01', '10']}, '59|81': {'11': ['01', '10']}, '59|82': {'11': ['01', '10']}, '59|83': {'11': ['01', '10']}, '59|84': {'11': ['01', '10']}, '59|85': {'11': ['01', '10']}, '59|86': {'11': ['01', '10']}, '59|87': {'11': ['01', '10']}, '59|88': {'11': ['01', '10']}, '59|89': {'11': ['01', '10']}, '59|90': {'11': ['01', '10']}, '59|91': {'11': ['01', '10']}, '59|92': {'11': ['01', '10']}, '59|93': {'11': ['01', '10']}, '59|94': {'11': ['01', '10']}, '59|95': {'11': ['01', '10']}, '59|96': {'11': ['01', '10']}, '59|97': {'11': ['01', '10']}, '59|98': {'11': ['01', '10']}, '59|99': {'11': ['01', '10']}, '60|99': {'00': ['10'], '11': ['10', '01']}, '60|98': {'00': ['10'], '11': ['10', '01']}, '60|97': {'00': ['10'], '11': ['10', '01']}, '61|99': {'00': ['10'], '11': ['10']}, '60|96': {'00': ['10'], '11': ['10', '01']}, '61|98': {'00': ['10'], '11': ['10']}, '60|95': {'00': ['10'], '11': ['10', '01']}, '61|97': {'00': ['10'], '11': ['10']}, '60|94': {'00': ['10'], '11': ['10', '01']}, '61|96': {'00': ['10'], '11': ['10']}, '60|93': {'00': ['10'], '11': ['10', '01']}, '61|95': {'00': ['10'], '11': ['10', '01']}, '60|92': {'00': ['10'], '11': ['10', '01']}, '61|94': {'00': ['10'], '11': ['10', '01']}, '60|91': {'00': ['10'], '11': ['10', '01']}, '61|93': {'00': ['10'], '11': ['10', '01']}, '62|95': {'00': ['10'], '11': ['10']}, '62|96': {'11': ['10']}, '62|97': {'11': ['10']}, '62|98': {'11': ['10']}, '62|99': {'11': ['10']}, '60|90': {'00': ['10'], '11': ['10', '01']}, '61|92': {'00': ['10'], '11': ['10', '01']}, '62|94': {'00': ['10'], '11': ['10']}, '60|89': {'00': ['10'], '11': ['10', '01']}, '61|91': {'00': ['10'], '11': ['10', '01']}, '62|93': {'00': ['10'], '11': ['10']}, '60|88': {'00': ['10'], '11': ['10', '01']}, '61|90': {'00': ['10'], '11': ['10', '01']}, '62|92': {'00': ['10'], '11': ['10']}, '60|87': {'00': ['10'], '11': ['10', '01']}, '61|89': {'00': ['10'], '11': ['10', '01']}, '62|91': {'00': ['10'], '11': ['10']}, '60|86': {'00': ['10'], '11': ['10', '01']}, '61|88': {'00': ['10'], '11': ['10', '01']}, '62|90': {'00': ['10'], '11': ['10']}, '60|85': {'00': ['10'], '11': ['10', '01']}, '61|87': {'00': ['10'], '11': ['10', '01']}, '62|89': {'00': ['10'], '11': ['10']}, '60|84': {'00': ['10'], '11': ['10', '01']}, '61|86': {'00': ['10'], '11': ['10', '01']}, '62|88': {'00': ['10'], '11': ['10']}, '60|83': {'00': ['10'], '11': ['10', '01']}, '61|85': {'00': ['10'], '11': ['10', '01']}, '62|87': {'00': ['10'], '11': ['10']}, '60|82': {'00': ['10'], '11': ['10', '01']}, '61|84': {'00': ['10'], '11': ['10', '01']}, '62|86': {'00': ['10'], '11': ['10']}, '60|81': {'00': ['10'], '11': ['10', '01']}, '61|83': {'00': ['10'], '11': ['10', '01']}, '62|85': {'00': ['10'], '11': ['10']}, '60|80': {'00': ['10'], '11': ['10', '01']}, '61|82': {'00': ['10'], '11': ['10', '01']}, '62|84': {'00': ['10'], '11': ['10']}, '60|79': {'00': ['10'], '11': ['10', '01']}, '61|81': {'00': ['10'], '11': ['10', '01']}, '62|83': {'00': ['10'], '11': ['10']}, '60|78': {'00': ['10'], '11': ['10', '01']}, '61|80': {'00': ['10'], '11': ['10', '01']}, '62|82': {'00': ['10'], '11': ['10']}, '60|77': {'00': ['10'], '11': ['10', '01']}, '61|79': {'00': ['10'], '11': ['10', '01']}, '62|81': {'00': ['10'], '11': ['10']}, '60|76': {'00': ['10'], '11': ['10', '01']}, '61|78': {'00': ['10'], '11': ['10', '01']}, '62|80': {'00': ['10'], '11': ['10']}, '60|75': {'00': ['10'], '11': ['10', '01']}, '61|77': {'00': ['10'], '11': ['10', '01']}, '62|79': {'00': ['10'], '11': ['10']}, '60|74': {'00': ['10'], '11': ['10', '01']}, '61|76': {'00': ['10'], '11': ['10', '01']}, '62|78': {'00': ['10'], '11': ['10']}, '60|73': {'00': ['10'], '11': ['10', '01']}, '61|75': {'00': ['10'], '11': ['10', '01']}, '62|77': {'00': ['10'], '11': ['10']}, '60|72': {'00': ['10'], '11': ['10', '01']}, '61|74': {'00': ['10'], '11': ['10', '01']}, '62|76': {'00': ['10'], '11': ['10']}, '60|71': {'00': ['10'], '11': ['10', '01']}, '61|73': {'00': ['10'], '11': ['10', '01']}, '62|75': {'00': ['10'], '11': ['10']}, '60|70': {'00': ['10'], '11': ['10', '01']}, '61|72': {'00': ['10'], '11': ['10', '01']}, '62|74': {'00': ['10'], '11': ['10']}, '60|69': {'00': ['10'], '11': ['10', '01']}, '61|71': {'00': ['10'], '11': ['10']}, '62|73': {'00': ['10'], '11': ['10']}, '60|68': {'00': ['10'], '11': ['10', '01']}, '61|70': {'00': ['10'], '11': ['10']}, '62|72': {'00': ['10']}, '60|67': {'00': ['10'], '11': ['10', '01']}, '61|69': {'00': ['10'], '11': ['10']}, '60|66': {'00': ['10'], '11': ['10', '01']}, '61|68': {'00': ['10'], '11': ['10']}, '60|65': {'00': ['10'], '11': ['10', '01']}, '61|67': {'00': ['10'], '11': ['10']}, '60|64': {'00': ['10'], '11': ['10', '01']}, '61|66': {'00': ['10'], '11': ['10']}, '60|63': {'00': ['10'], '11': ['10', '01']}, '61|65': {'00': ['10'], '11': ['10']}, '59|60': {'00': ['10'], '11': ['10', '01']}, '60|62': {'00': ['10'], '11': ['10', '01']}, '61|64': {'00': ['10'], '11': ['10']}, '58|57': {'00': ['10'], '11': ['10']}, '59|59': {'00': ['10'], '11': ['10']}, '60|61': {'00': ['10'], '11': ['10']}, '61|63': {'00': ['10'], '11': ['10']}, '56|53': {'00': ['10']}, '57|54': {'00': ['10']}, '58|56': {'00': ['10']}, '59|58': {'00': ['10']}, '60|60': {'00': ['10']}, '61|62': {'00': ['10']}, '49|44': {'01': ['10'], '11': ['00', '10']}, '50|45': {'01': ['10'], '11': ['00', '10']}, '51|46': {'01': ['10'], '11': ['10', '00']}, '52|48': {'01': ['10'], '11': ['10', '00']}, '48|42': {'01': ['10'], '11': ['00', '10']}, '45|37': {'01': ['10']}, '45|38': {'11': ['00']}, '46|38': {'01': ['10']}, '46|39': {'11': ['00']}, '47|39': {'01': ['10']}, '47|40': {'11': ['00', '10']}, '48|41': {'01': ['10'], '11': ['10']}, '49|43': {'01': ['10'], '11': ['10', '00']}, '50|44': {'01': ['10'], '11': ['10']}, '52|47': {'01': ['10'], '11': ['10']}, '51|45': {'01': ['10']}, '49|42': {'01': ['10']}, '48|40': {'01': ['10']}, '50|43': {'01': ['10']}, '52|46': {'01': ['10']}, '53|49': {'01': ['10'], '11': ['10']}, '53|48': {'01': ['10']}, '54|50': {'00': ['10']}}

#from 43,25 F 01
##visitedF = {'43|25': {'01': ['00', '11']}, '43|26': {'10': ['00']}, '44|26': {'01': ['10']}, '44|27': {'11': ['00', '11']}, '44|28': {'10': ['00']}, '45|28': {'01': ['10'], '11': ['00']}, '45|29': {'11': ['00']}, '46|29': {'01': ['10'], '11': ['00']}, '46|30': {'11': ['00', '10']}, '46|31': {'11': ['00']}, '47|31': {'01': ['10'], '11': ['00', '10']}, '47|32': {'11': ['00', '10']}, '47|33': {'11': ['00']}, '48|33': {'01': ['10'], '11': ['00', '10']}, '48|34': {'11': ['00', '10']}, '48|35': {'11': ['00']}, '49|35': {'01': ['10'], '11': ['00', '10']}, '49|36': {'11': ['00', '10']}, '49|37': {'11': ['00']}, '50|37': {'01': ['10'], '11': ['00', '10']}, '50|38': {'11': ['00', '10']}, '50|39': {'11': ['00', '10']}, '50|40': {'11': ['00', '10']}, '50|41': {'11': ['00']}, '51|41': {'01': ['10'], '11': ['00', '10']}, '51|42': {'11': ['00', '10']}, '51|43': {'11': ['00', '10']}, '51|44': {'11': ['00']}, '52|44': {'01': ['10'], '11': ['00', '10']}, '52|45': {'11': ['00', '10']}, '52|46': {'11': ['00', '10']}, '52|47': {'11': ['00', '10']}, '52|48': {'11': ['00', '10']}, '52|49': {'11': ['00', '10']}, '52|50': {'11': ['10']}, '52|51': {'11': []}, '53|49': {'01': ['10'], '11': ['10', '00']}, '53|50': {'11': ['10', '01']}, '53|51': {'11': ['01']}, '54|51': {'00': ['10'], '11': ['10', '01']}, '54|52': {'11': ['01']}, '55|52': {'00': ['10'], '11': ['10', '01']}, '55|53': {'11': ['01', '10']}, '55|54': {'11': ['01']}, '56|54': {'00': ['10'], '11': ['01', '10']}, '56|55': {'11': ['01']}, '57|55': {'00': ['10'], '11': ['10', '01']}, '57|56': {'11': ['10', '01']}, '57|57': {'11': ['01', '10']}, '57|58': {'11': ['01']}, '58|58': {'00': ['10'], '11': ['10', '01']}, '58|59': {'11': ['10', '01']}, '58|60': {'11': ['01', '10']}, '58|61': {'11': ['01']}, '59|61': {'00': ['10'], '11': ['10', '01']}, '59|62': {'11': ['10', '01']}, '59|63': {'11': ['01', '10']}, '59|64': {'11': ['01', '10']}, '59|65': {'11': ['01', '10']}, '59|66': {'11': ['01', '10']}, '59|67': {'11': ['01', '10']}, '59|68': {'11': ['01', '10']}, '59|69': {'11': ['01', '10']}, '59|70': {'11': ['01', '10']}, '59|71': {'11': ['01', '10']}, '59|72': {'11': ['01', '10']}, '59|73': {'11': ['01', '10']}, '59|74': {'11': ['01', '10']}, '59|75': {'11': ['01', '10']}, '59|76': {'11': ['01', '10']}, '59|77': {'11': ['01', '10']}, '59|78': {'11': ['01', '10']}, '59|79': {'11': ['01', '10']}, '59|80': {'11': ['01', '10']}, '59|81': {'11': ['01', '10']}, '59|82': {'11': ['01', '10']}, '59|83': {'11': ['01', '10']}, '59|84': {'11': ['01', '10']}, '59|85': {'11': ['01', '10']}, '59|86': {'11': ['01', '10']}, '59|87': {'11': ['01', '10']}, '59|88': {'11': ['01', '10']}, '59|89': {'11': ['01', '10']}, '59|90': {'11': ['01', '10']}, '59|91': {'11': ['01', '10']}, '59|92': {'11': ['01', '10']}, '59|93': {'11': ['01', '10']}, '59|94': {'11': ['01', '10']}, '59|95': {'11': ['01', '10']}, '59|96': {'11': ['01', '10']}, '59|97': {'11': ['01', '10']}, '59|98': {'11': ['01', '10']}, '59|99': {'11': ['01', '10']}, '60|99': {'00': ['10'], '11': ['10', '01']}, '60|98': {'00': ['10'], '11': ['10', '01']}, '60|97': {'00': ['10'], '11': ['10', '01']}, '61|99': {'00': ['10'], '11': ['10']}, '60|96': {'00': ['10'], '11': ['10', '01']}, '61|98': {'00': ['10'], '11': ['10']}, '60|95': {'00': ['10'], '11': ['10', '01']}, '61|97': {'00': ['10'], '11': ['10']}, '60|94': {'00': ['10'], '11': ['10', '01']}, '61|96': {'00': ['10'], '11': ['10']}, '60|93': {'00': ['10'], '11': ['10', '01']}, '61|95': {'00': ['10'], '11': ['10', '01']}, '60|92': {'00': ['10'], '11': ['10', '01']}, '61|94': {'00': ['10'], '11': ['10', '01']}, '60|91': {'00': ['10'], '11': ['10', '01']}, '61|93': {'00': ['10'], '11': ['10', '01']}, '62|95': {'00': ['10'], '11': ['10']}, '62|96': {'11': ['10']}, '62|97': {'11': ['10']}, '62|98': {'11': ['10']}, '62|99': {'11': ['10']}, '60|90': {'00': ['10'], '11': ['10', '01']}, '61|92': {'00': ['10'], '11': ['10', '01']}, '62|94': {'00': ['10'], '11': ['10']}, '60|89': {'00': ['10'], '11': ['10', '01']}, '61|91': {'00': ['10'], '11': ['10', '01']}, '62|93': {'00': ['10'], '11': ['10']}, '60|88': {'00': ['10'], '11': ['10', '01']}, '61|90': {'00': ['10'], '11': ['10', '01']}, '62|92': {'00': ['10'], '11': ['10']}, '60|87': {'00': ['10'], '11': ['10', '01']}, '61|89': {'00': ['10'], '11': ['10', '01']}, '62|91': {'00': ['10'], '11': ['10']}, '60|86': {'00': ['10'], '11': ['10', '01']}, '61|88': {'00': ['10'], '11': ['10', '01']}, '62|90': {'00': ['10'], '11': ['10']}, '60|85': {'00': ['10'], '11': ['10', '01']}, '61|87': {'00': ['10'], '11': ['10', '01']}, '62|89': {'00': ['10'], '11': ['10']}, '60|84': {'00': ['10'], '11': ['10', '01']}, '61|86': {'00': ['10'], '11': ['10', '01']}, '62|88': {'00': ['10'], '11': ['10']}, '60|83': {'00': ['10'], '11': ['10', '01']}, '61|85': {'00': ['10'], '11': ['10', '01']}, '62|87': {'00': ['10'], '11': ['10']}, '60|82': {'00': ['10'], '11': ['10', '01']}, '61|84': {'00': ['10'], '11': ['10', '01']}, '62|86': {'00': ['10'], '11': ['10']}, '60|81': {'00': ['10'], '11': ['10', '01']}, '61|83': {'00': ['10'], '11': ['10', '01']}, '62|85': {'00': ['10'], '11': ['10']}, '60|80': {'00': ['10'], '11': ['10', '01']}, '61|82': {'00': ['10'], '11': ['10', '01']}, '62|84': {'00': ['10'], '11': ['10']}, '60|79': {'00': ['10'], '11': ['10', '01']}, '61|81': {'00': ['10'], '11': ['10', '01']}, '62|83': {'00': ['10'], '11': ['10']}, '60|78': {'00': ['10'], '11': ['10', '01']}, '61|80': {'00': ['10'], '11': ['10', '01']}, '62|82': {'00': ['10'], '11': ['10']}, '60|77': {'00': ['10'], '11': ['10', '01']}, '61|79': {'00': ['10'], '11': ['10', '01']}, '62|81': {'00': ['10'], '11': ['10']}, '60|76': {'00': ['10'], '11': ['10', '01']}, '61|78': {'00': ['10'], '11': ['10', '01']}, '62|80': {'00': ['10'], '11': ['10']}, '60|75': {'00': ['10'], '11': ['10', '01']}, '61|77': {'00': ['10'], '11': ['10', '01']}, '62|79': {'00': ['10'], '11': ['10']}, '60|74': {'00': ['10'], '11': ['10', '01']}, '61|76': {'00': ['10'], '11': ['10', '01']}, '62|78': {'00': ['10'], '11': ['10']}, '60|73': {'00': ['10'], '11': ['10', '01']}, '61|75': {'00': ['10'], '11': ['10', '01']}, '62|77': {'00': ['10'], '11': ['10']}, '60|72': {'00': ['10'], '11': ['10', '01']}, '61|74': {'00': ['10'], '11': ['10', '01']}, '62|76': {'00': ['10'], '11': ['10']}, '60|71': {'00': ['10'], '11': ['10', '01']}, '61|73': {'00': ['10'], '11': ['10', '01']}, '62|75': {'00': ['10'], '11': ['10']}, '60|70': {'00': ['10'], '11': ['10', '01']}, '61|72': {'00': ['10'], '11': ['10', '01']}, '62|74': {'00': ['10'], '11': ['10']}, '60|69': {'00': ['10'], '11': ['10', '01']}, '61|71': {'00': ['10'], '11': ['10']}, '62|73': {'00': ['10'], '11': ['10']}, '60|68': {'00': ['10'], '11': ['10', '01']}, '61|70': {'00': ['10'], '11': ['10']}, '62|72': {'00': ['10'], '11': ['10']}, '60|67': {'00': ['10'], '11': ['10', '01']}, '61|69': {'00': ['10'], '11': ['10']}, '60|66': {'00': ['10'], '11': ['10', '01']}, '61|68': {'00': ['10'], '11': ['10']}, '60|65': {'00': ['10'], '11': ['10', '01']}, '61|67': {'00': ['10'], '11': ['10']}, '60|64': {'00': ['10'], '11': ['10', '01']}, '61|66': {'00': ['10'], '11': ['10']}, '60|63': {'00': ['10'], '11': ['10', '01']}, '61|65': {'00': ['10'], '11': ['10']}, '59|60': {'00': ['10'], '11': ['10', '01']}, '60|62': {'00': ['10'], '11': ['10', '01']}, '61|64': {'00': ['10'], '11': ['10']}, '58|57': {'00': ['10'], '11': ['10', '01']}, '59|59': {'00': ['10'], '11': ['10', '01']}, '60|61': {'00': ['10'], '11': ['10', '01']}, '61|63': {'00': ['10'], '11': ['10']}, '56|53': {'00': ['10'], '11': ['10', '01']}, '57|54': {'00': ['10'], '11': ['10', '01']}, '58|56': {'00': ['10'], '11': ['10', '01']}, '59|58': {'00': ['10'], '11': ['10', '01']}, '60|60': {'00': ['10'], '11': ['10', '01']}, '61|62': {'00': ['10'], '11': ['10']}, '53|48': {'01': ['10'], '11': ['10', '00']}, '54|50': {'00': ['10'], '11': ['10', '01']}, '53|47': {'01': ['10'], '11': ['00', '10']}, '53|46': {'01': ['10'], '11': ['10', '00']}, '54|49': {'01': ['10'], '11': ['10', '00']}, '55|51': {'00': ['10'], '11': ['10', '01']}, '54|48': {'01': ['10'], '11': ['10', '00']}, '55|50': {'00': ['10'], '11': ['10', '01']}, '56|52': {'00': ['10'], '11': ['10', '01']}, '54|47': {'01': ['10'], '11': ['10', '00']}, '53|45': {'01': ['10'], '11': ['10', '00']}, '52|43': {'01': ['10'], '11': ['10', '00']}, '54|46': {'01': ['10'], '11': ['10', '00']}, '55|49': {'01': ['10'], '11': ['10', '00']}, '56|51': {'00': ['10'], '11': ['10', '01']}, '57|53': {'00': ['10'], '11': ['10', '01']}, '58|55': {'00': ['10'], '11': ['10', '01']}, '59|57': {'00': ['10'], '11': ['10', '01']}, '60|59': {'00': ['10'], '11': ['10', '01']}, '61|61': {'00': ['10'], '11': ['10']}, '55|48': {'01': ['10'], '11': ['10', '00']}, '56|50': {'00': ['10'], '11': ['01', '10']}, '57|52': {'00': ['10'], '11': ['10', '01']}, '58|54': {'00': ['10'], '11': ['10', '01']}, '59|56': {'00': ['10'], '11': ['10', '01']}, '60|58': {'00': ['10'], '11': ['10', '01']}, '61|60': {'00': ['10'], '11': ['10', '01']}, '53|44': {'01': ['10'], '11': ['10', '00']}, '55|47': {'01': ['10'], '11': ['10', '00']}, '52|42': {'01': ['10'], '11': ['10', '00']}, '54|45': {'01': ['10'], '11': ['00', '10']}, '51|40': {'01': ['10'], '11': ['00', '10']}, '53|43': {'01': ['10'], '11': ['10', '00']}, '55|46': {'01': ['10'], '11': ['10', '00']}, '56|49': {'01': ['10'], '11': ['10', '00']}, '57|51': {'00': ['10'], '11': ['10']}, '58|53': {'00': ['10'], '11': ['10', '01']}, '59|55': {'00': ['10'], '11': ['10', '01']}, '60|57': {'00': ['10'], '11': ['10', '01']}, '61|59': {'00': ['10'], '11': ['10', '01']}, '57|50': {'00': ['10'], '11': ['10']}, '56|48': {'01': ['10'], '11': ['10', '00']}, '52|41': {'01': ['10'], '11': ['10', '00']}, '51|39': {'01': ['10'], '11': ['10', '00']}, '53|42': {'01': ['10'], '11': ['10', '00']}, '54|44': {'01': ['10'], '11': ['10', '00']}, '56|47': {'01': ['10'], '11': ['10', '00']}, '55|45': {'01': ['10'], '11': ['00', '10']}, '57|49': {'01': ['10'], '11': ['10', '00']}, '58|52': {'00': ['10'], '11': ['01', '10']}, '59|54': {'00': ['10'], '11': ['10', '01']}, '60|56': {'00': ['10'], '11': ['10', '01']}, '61|58': {'00': ['10'], '11': ['10', '01']}, '52|40': {'01': ['10'], '11': ['10', '00']}, '54|43': {'01': ['10'], '11': ['00', '10']}, '56|46': {'01': ['10'], '11': ['00', '10']}, '57|48': {'01': ['10'], '11': ['10', '00']}, '51|38': {'01': ['10'], '11': ['10', '00']}, '53|41': {'01': ['10'], '11': ['10', '00']}, '50|36': {'01': ['10'], '11': ['00', '10']}, '52|39': {'01': ['10'], '11': ['10', '00']}, '54|42': {'01': ['10'], '11': ['00', '10']}, '55|44': {'01': ['10'], '11': ['00', '10']}, '57|47': {'01': ['10'], '11': ['00', '10']}, '56|45': {'01': ['10'], '11': ['00', '10']}, '58|49': {'01': ['10'], '11': ['10', '00']}, '58|50': {'11': ['01', '10']}, '58|51': {'11': ['01', '10']}, '59|53': {'00': ['10'], '11': ['10', '01']}, '60|55': {'00': ['10'], '11': ['10', '01']}, '61|57': {'00': ['10'], '11': ['10', '01']}, '59|52': {'00': ['10'], '11': ['10']}, '60|54': {'00': ['10'], '11': ['10']}, '61|56': {'00': ['10'], '11': ['10', '01']}, '59|51': {'00': ['10'], '11': ['10', '01']}, '59|50': {'00': ['10'], '11': ['01', '10']}, '57|46': {'01': ['10'], '11': ['00', '10']}, '58|48': {'01': ['10'], '11': ['10', '00']}, '58|47': {'01': ['10'], '11': ['00', '10']}, '59|49': {'01': ['10'], '11': ['10', '00']}, '60|53': {'00': ['10'], '11': ['10', '01']}, '60|51': {'00': ['10'], '11': ['10', '01']}, '60|52': {'11': ['10']}, '60|50': {'00': ['10'], '11': ['01', '10']}, '55|43': {'01': ['10'], '11': ['00', '10']}, '56|44': {'01': ['10'], '11': ['00']}, '57|45': {'01': ['00', '10'], '11': ['00']}, '58|46': {'01': ['00', '10'], '11': ['00']}, '59|48': {'01': ['10'], '11': ['00', '10']}, '59|47': {'01': ['10']}, '60|49': {'01': ['10'], '11': ['10', '00']}, '61|55': {'00': ['10'], '11': ['10', '01']}, '61|53': {'00': ['10'], '11': ['10']}, '61|54': {'11': ['10', '01']}, '61|51': {'00': ['10'], '11': ['10', '01']}, '61|52': {'11': ['10', '01']}, '61|50': {'00': ['10'], '11': ['10']}, '60|48': {'01': ['10'], '11': ['00', '10']}, '59|46': {'01': ['00'], '10': ['00']}, '60|46': {'01': ['00', '11'], '10': ['00']}, '61|46': {'01': ['11'], '10': ['00', '11']}, '61|47': {'10': ['00']}, '62|47': {'01': ['00'], '10': ['00']}, '63|47': {'01': ['11'], '10': ['00', '11']}, '63|48': {'10': ['00']}, '64|48': {'01': ['00'], '10': ['00']}, '65|48': {'01': ['00']}, '66|48': {'01': ['00']}, '67|48': {'01': ['00']}, '68|48': {'01': ['00']}, '69|48': {'01': ['00']}, '70|48': {'01': ['00']}, '71|48': {'01': ['11']}, '71|49': {'10': ['10']}, '71|50': {'11': ['10']}, '71|51': {'11': ['10']}, '71|52': {'11': ['10']}, '71|53': {'11': ['10']}, '71|54': {'11': ['10']}, '71|55': {'11': ['10']}, '71|56': {'11': ['10']}, '71|57': {'11': ['10']}, '71|58': {'11': ['10']}, '71|59': {'11': ['10']}, '71|60': {'11': ['10']}, '71|61': {'11': ['10']}, '71|62': {'11': ['10']}, '71|63': {'11': ['10']}, '71|64': {'11': ['10']}, '71|65': {'11': ['10']}, '71|66': {'11': ['10']}, '71|67': {'11': ['10']}, '71|68': {'11': ['10']}, '71|69': {'11': ['10']}, '71|70': {'11': ['10']}, '71|71': {'11': ['10']}, '71|72': {'11': ['10']}, '71|73': {'11': ['10']}, '71|74': {'11': ['10']}, '71|75': {'11': ['10']}, '71|76': {'11': ['10']}, '71|77': {'11': ['10']}, '71|78': {'11': ['10']}, '71|79': {'11': ['10']}, '71|80': {'11': ['10']}, '71|81': {'11': ['10']}, '71|82': {'11': ['10']}, '71|83': {'11': ['10']}, '71|84': {'11': ['10']}, '71|85': {'11': ['10']}, '71|86': {'11': ['10']}, '71|87': {'11': ['10']}, '71|88': {'11': ['10']}, '71|89': {'11': ['10']}, '71|90': {'11': ['10']}, '71|91': {'11': ['10']}, '71|92': {'11': ['10']}, '71|93': {'11': ['10']}, '71|94': {'11': ['10']}, '71|95': {'11': ['10']}, '71|96': {'11': ['10']}, '71|97': {'11': ['10']}, '71|98': {'11': ['10']}, '71|99': {'11': ['10']}, '58|45': {'01': ['00', '10'], '10': ['10']}, '60|47': {'10': ['10']}, '61|49': {'01': ['10'], '11': ['10']}, '62|60': {'00': ['10'], '11': ['10']}, '62|61': {'11': ['10']}, '62|62': {'11': ['10']}, '62|63': {'11': ['10']}, '62|64': {'11': ['10']}, '62|65': {'11': ['10']}, '62|66': {'11': ['10']}, '62|67': {'11': ['10']}, '62|68': {'11': ['10']}, '62|69': {'11': ['10']}, '62|70': {'11': ['10']}, '62|71': {'11': ['10']}, '62|59': {'00': ['10'], '11': ['10']}, '62|58': {'00': ['10'], '11': ['10']}, '62|57': {'00': ['10'], '11': ['10']}, '62|56': {'00': ['10'], '11': ['10']}, '62|55': {'00': ['10'], '11': ['10']}, '62|54': {'00': ['10'], '11': ['10']}, '62|52': {'00': ['10'], '11': ['10']}, '62|53': {'11': ['10']}, '62|51': {'00': ['10']}, '61|48': {'01': ['10']}, '59|45': {'01': ['00', '11'], '10': ['00']}, '60|45': {'01': ['11'], '10': ['00', '11']}, '51|37': {'01': ['10'], '11': ['10', '00']}, '53|40': {'01': ['10'], '11': ['10', '00']}, '49|34': {'01': ['10'], '11': ['00', '10']}, '50|35': {'01': ['10'], '11': ['00', '10']}, '52|38': {'01': ['10'], '11': ['10', '00']}, '54|41': {'01': ['10'], '11': ['00', '10']}, '55|42': {'01': ['10'], '11': ['00', '10']}, '56|43': {'01': ['10', '00'], '11': ['00']}, '57|44': {'01': ['00', '10'], '11': ['00']}, '58|44': {'01': ['00', '11'], '10': ['00']}, '59|44': {'01': ['00', '11'], '10': ['00']}, '60|44': {'01': ['11'], '10': ['00', '11']}, '61|45': {'01': ['11'], '10': ['11', '00']}, '62|46': {'01': ['11'], '10': ['11', '00']}, '51|36': {'01': ['10'], '11': ['10', '00']}, '53|39': {'01': ['10'], '11': ['10', '00']}, '57|43': {'01': ['00', '10'], '11': ['00']}, '58|43': {'01': ['00', '11'], '10': ['00']}, '59|43': {'01': ['11'], '10': ['00']}, '48|32': {'01': ['10'], '11': ['00', '10']}, '49|33': {'01': ['10'], '11': ['00', '10']}, '50|34': {'01': ['10'], '11': ['00', '10']}, '52|37': {'01': ['10'], '11': ['10', '00']}, '54|40': {'01': ['10'], '11': ['00', '10']}, '55|41': {'01': ['10'], '11': ['00', '10']}, '56|42': {'01': ['10', '00'], '11': ['00']}, '51|35': {'01': ['10'], '11': ['10', '00']}, '53|38': {'01': ['10'], '11': ['10', '00']}, '57|42': {'01': ['00', '10']}, '58|42': {'01': ['00', '11'], '10': ['00']}, '59|42': {'01': ['11'], '10': ['00', '11']}, '60|43': {'01': ['11'], '10': ['11', '00']}, '61|44': {'01': ['11'], '10': ['11', '00']}, '47|30': {'01': ['10'], '11': ['00', '10']}, '48|31': {'01': ['10'], '11': ['00', '10']}, '49|32': {'01': ['10'], '11': ['00', '10']}, '50|33': {'01': ['10'], '11': ['00', '10']}, '52|36': {'01': ['10'], '11': ['10', '00']}, '54|39': {'01': ['10'], '11': ['00', '10']}, '55|40': {'01': ['10'], '11': ['00', '10']}, '56|41': {'01': ['10', '00'], '11': ['00']}, '51|34': {'01': ['10'], '11': ['10', '00']}, '53|37': {'01': ['10'], '11': ['00', '10']}, '57|41': {'01': ['00']}, '58|41': {'01': ['00', '11'], '10': ['00']}, '59|41': {'01': ['11'], '10': ['00', '11']}, '60|42': {'01': ['11'], '10': ['11', '00']}, '45|27': {'01': ['00', '10']}, '46|28': {'01': ['10'], '11': ['00']}, '47|29': {'01': ['10'], '11': ['00', '10']}, '48|30': {'01': ['10'], '11': ['10', '00']}, '49|31': {'01': ['10'], '11': ['00', '10']}, '50|32': {'01': ['10'], '11': ['00', '10']}, '52|35': {'01': ['10'], '11': ['10', '00']}, '54|38': {'01': ['10'], '11': ['00', '10']}, '55|39': {'01': ['10'], '11': ['00', '10']}, '56|40': {'01': ['10', '00'], '11': ['00']}, '51|33': {'01': ['10'], '11': ['10', '00']}, '53|36': {'01': ['10'], '11': ['00', '10']}, '54|37': {'01': ['10'], '11': ['00', '10']}, '57|40': {'01': ['00'], '10': ['00'], '11': ['00']}, '58|40': {'01': ['00', '11'], '10': ['00', '11']}, '59|40': {'01': ['11'], '10': ['00', '11']}, '61|43': {'01': ['11'], '10': ['11', '00']}, '62|45': {'01': ['11'], '10': ['11', '00']}, '60|41': {'01': ['11'], '10': ['11', '00']}, '55|38': {'01': ['10', '00'], '11': ['00']}, '56|39': {'01': ['00', '10'], '11': ['00']}, '57|39': {'01': ['00', '11'], '10': ['00', '10']}, '58|39': {'01': ['00', '11'], '10': ['00']}, '59|39': {'01': ['11'], '10': ['11', '00']}, '61|42': {'01': ['11'], '10': ['11']}, '62|44': {'01': ['11'], '10': ['11']}, '63|46': {'01': ['11'], '10': ['11']}, '64|47': {'01': ['11']}, '60|40': {'01': ['11'], '10': ['11']}, '46|27': {'01': ['10'], '11': ['00']}, '47|28': {'01': ['10'], '11': ['00', '10']}, '48|29': {'01': ['10'], '11': ['00', '10']}, '44|25': {'01': ['00']}, '45|25': {'01': ['00', '10']}, '45|26': {'11': ['00']}, '46|26': {'01': ['10'], '11': ['00']}, '47|27': {'01': ['10'], '11': ['10']}, '49|30': {'01': ['10'], '11': ['00', '10']}, '50|31': {'01': ['10'], '11': ['00', '10']}, '52|34': {'01': ['10'], '11': ['10', '00']}, '51|32': {'01': ['10'], '11': ['10']}, '53|35': {'01': ['10'], '11': ['00', '10']}, '54|36': {'01': ['10'], '11': ['00', '10']}, '56|38': {'01': ['00', '10'], '11': ['00']}, '57|38': {'01': ['00', '11']}, '58|38': {'01': ['00', '11'], '10': ['00']}, '59|38': {'01': ['11']}, '61|41': {'01': ['11']}, '62|43': {'01': ['11']}, '63|45': {'01': ['11']}, '55|37': {'01': ['10'], '11': ['00']}, '48|28': {'01': ['10']}, '49|29': {'01': ['10']}, '50|30': {'01': ['10']}, '52|33': {'01': ['10']}, '51|31': {'01': ['10']}, '53|34': {'01': ['10']}, '54|35': {'01': ['10']}, '55|36': {'01': ['10']}, '56|37': {'01': ['00', '10']}, '57|37': {'01': ['00']}, '58|37': {'01': ['11']}, '60|39': {'01': ['11']}, '46|25': {'01': ['10']}, '47|26': {'01': ['10']}}
#from 43,25 B 11
##visitedB = {'43|25': {'11': ['01']}, '44|25': {'00': ['01', '11']}, '44|26': {'10': ['01']}, '45|26': {'00': ['01'], '10': ['01']}, '46|26': {'00': ['11']}, '46|27': {'10': ['11', '01']}, '46|28': {'10': ['01']}, '47|28': {'00': ['11'], '10': ['01', '11']}, '47|29': {'10': ['01', '11']}, '47|30': {'10': ['01', '11']}, '47|31': {'10': ['01']}, '48|31': {'00': ['11'], '10': ['01', '11']}, '48|32': {'10': ['01', '11']}, '48|33': {'10': ['01', '11']}, '48|34': {'10': ['01']}, '49|34': {'00': ['11'], '10': ['01', '11']}, '49|35': {'10': ['01', '11']}, '49|36': {'10': ['01', '11']}, '49|37': {'10': ['01']}, '50|37': {'00': ['11'], '10': ['11', '01']}, '50|38': {'10': ['11', '01']}, '50|39': {'10': ['01', '11']}, '50|40': {'10': ['01']}, '51|40': {'00': ['11'], '10': ['11', '01']}, '51|41': {'10': ['11', '01']}, '51|42': {'10': ['01', '11']}, '51|43': {'10': ['01']}, '52|43': {'00': ['11'], '10': ['11', '01']}, '52|44': {'10': ['11', '01']}, '52|45': {'10': ['01', '11']}, '52|46': {'10': ['01']}, '53|46': {'00': ['11'], '10': ['11', '01']}, '53|47': {'10': ['01', '11']}, '53|48': {'10': ['01', '11']}, '53|49': {'10': ['01', '11']}, '53|50': {'10': ['11']}, '53|51': {'10': ['00']}, '54|51': {'01': ['11'], '10': ['00', '11']}, '54|52': {'10': ['00']}, '55|52': {'01': ['11'], '10': ['11', '00']}, '55|53': {'10': ['00', '11']}, '55|54': {'10': ['00']}, '56|54': {'01': ['11'], '10': ['00', '11']}, '56|55': {'10': ['00']}, '57|55': {'01': ['11'], '10': ['11', '00']}, '57|56': {'10': ['11', '00']}, '57|57': {'10': ['00']}, '58|57': {'01': ['11'], '10': ['11', '00']}, '58|58': {'10': ['11', '00']}, '58|59': {'10': ['00', '11']}, '58|60': {'10': ['00', '11']}, '58|61': {'10': ['00', '11']}, '58|62': {'10': ['00', '11']}, '58|63': {'10': ['00', '11']}, '58|64': {'10': ['00', '11']}, '58|65': {'10': ['00']}, '59|65': {'01': ['11'], '10': ['11', '00']}, '59|66': {'10': ['11', '00']}, '59|67': {'10': ['00', '11']}, '59|68': {'10': ['00', '11']}, '59|69': {'10': ['00', '11']}, '59|70': {'10': ['00', '11']}, '59|71': {'10': ['00', '11']}, '59|72': {'10': ['00', '11']}, '59|73': {'10': ['00', '11']}, '59|74': {'10': ['00', '11']}, '59|75': {'10': ['00', '11']}, '59|76': {'10': ['00', '11']}, '59|77': {'10': ['00', '11']}, '59|78': {'10': ['00', '11']}, '59|79': {'10': ['00', '11']}, '59|80': {'10': ['00', '11']}, '59|81': {'10': ['00', '11']}, '59|82': {'10': ['00', '11']}, '59|83': {'10': ['00', '11']}, '59|84': {'10': ['00', '11']}, '59|85': {'10': ['00', '11']}, '59|86': {'10': ['00', '11']}, '59|87': {'10': ['00', '11']}, '59|88': {'10': ['00', '11']}, '59|89': {'10': ['00', '11']}, '59|90': {'10': ['00', '11']}, '59|91': {'10': ['00', '11']}, '59|92': {'10': ['00', '11']}, '59|93': {'10': ['00', '11']}, '59|94': {'10': ['00', '11']}, '59|95': {'10': ['00', '11']}, '59|96': {'10': ['00', '11']}, '59|97': {'10': ['00', '11']}, '59|98': {'10': ['00', '11']}, '59|99': {'10': ['00', '11']}, '60|99': {'01': ['11'], '10': ['11', '00']}, '60|98': {'01': ['11'], '10': ['11', '00']}, '60|97': {'01': ['11'], '10': ['11', '00']}, '60|96': {'01': ['11'], '10': ['11', '00']}, '61|99': {'01': ['11'], '10': ['11', '00']}, '60|95': {'01': ['11'], '10': ['11', '00']}, '61|98': {'01': ['11'], '10': ['11', '00']}, '60|94': {'01': ['11'], '10': ['11', '00']}, '61|97': {'01': ['11'], '10': ['11', '00']}, '60|93': {'01': ['11'], '10': ['11', '00']}, '61|96': {'01': ['11'], '10': ['11', '00']}, '62|99': {'01': ['11'], '10': ['11', '00']}, '60|92': {'01': ['11'], '10': ['11', '00']}, '61|95': {'01': ['11'], '10': ['11', '00']}, '62|98': {'01': ['11'], '10': ['11', '00']}, '60|91': {'01': ['11'], '10': ['11', '00']}, '61|94': {'01': ['11'], '10': ['11', '00']}, '62|97': {'01': ['11'], '10': ['11', '00']}, '60|90': {'01': ['11'], '10': ['11', '00']}, '61|93': {'01': ['11'], '10': ['11', '00']}, '62|96': {'01': ['11'], '10': ['11', '00']}, '63|99': {'01': ['11'], '10': ['11', '00']}, '60|89': {'01': ['11'], '10': ['11', '00']}, '61|92': {'01': ['11'], '10': ['11', '00']}, '62|95': {'01': ['11'], '10': ['11', '00']}, '63|98': {'01': ['11'], '10': ['11', '00']}, '60|88': {'01': ['11'], '10': ['11', '00']}, '61|91': {'01': ['11'], '10': ['11', '00']}, '62|94': {'01': ['11'], '10': ['11', '00']}, '63|97': {'01': ['11'], '10': ['11', '00']}, '60|87': {'01': ['11'], '10': ['11', '00']}, '61|90': {'01': ['11'], '10': ['11', '00']}, '62|93': {'01': ['11'], '10': ['11', '00']}, '63|96': {'01': ['11'], '10': ['11', '00']}, '64|99': {'01': ['11'], '10': ['11', '00']}, '60|86': {'01': ['11'], '10': ['11', '00']}, '61|89': {'01': ['11'], '10': ['11', '00']}, '62|92': {'01': ['11'], '10': ['11', '00']}, '63|95': {'01': ['11'], '10': ['11', '00']}, '64|98': {'01': ['11'], '10': ['11', '00']}, '60|85': {'01': ['11'], '10': ['11', '00']}, '61|88': {'01': ['11'], '10': ['11', '00']}, '62|91': {'01': ['11'], '10': ['11', '00']}, '63|94': {'01': ['11'], '10': ['11', '00']}, '64|97': {'01': ['11'], '10': ['11', '00']}, '60|84': {'01': ['11'], '10': ['11', '00']}, '61|87': {'01': ['11'], '10': ['11', '00']}, '62|90': {'01': ['11'], '10': ['11', '00']}, '63|93': {'01': ['11'], '10': ['11', '00']}, '64|96': {'01': ['11'], '10': ['11', '00']}, '65|99': {'01': ['11'], '10': ['11', '00']}, '60|83': {'01': ['11'], '10': ['11', '00']}, '61|86': {'01': ['11'], '10': ['11', '00']}, '62|89': {'01': ['11'], '10': ['11', '00']}, '63|92': {'01': ['11'], '10': ['11', '00']}, '64|95': {'01': ['11'], '10': ['11', '00']}, '65|98': {'01': ['11'], '10': ['11', '00']}, '60|82': {'01': ['11'], '10': ['11', '00']}, '61|85': {'01': ['11'], '10': ['11', '00']}, '62|88': {'01': ['11'], '10': ['11', '00']}, '63|91': {'01': ['11'], '10': ['11', '00']}, '64|94': {'01': ['11'], '10': ['11', '00']}, '65|97': {'01': ['11'], '10': ['11', '00']}, '60|81': {'01': ['11'], '10': ['11', '00']}, '61|84': {'01': ['11'], '10': ['11', '00']}, '62|87': {'01': ['11'], '10': ['11', '00']}, '63|90': {'01': ['11'], '10': ['11', '00']}, '64|93': {'01': ['11'], '10': ['11', '00']}, '65|96': {'01': ['11'], '10': ['11', '00']}, '66|99': {'01': ['11'], '10': ['11', '00']}, '60|80': {'01': ['11'], '10': ['11', '00']}, '61|83': {'01': ['11'], '10': ['11', '00']}, '62|86': {'01': ['11'], '10': ['11', '00']}, '63|89': {'01': ['11'], '10': ['11', '00']}, '64|92': {'01': ['11'], '10': ['11', '00']}, '65|95': {'01': ['11'], '10': ['11', '00']}, '66|98': {'01': ['11'], '10': ['11', '00']}, '60|79': {'01': ['11'], '10': ['11', '00']}, '61|82': {'01': ['11'], '10': ['11', '00']}, '62|85': {'01': ['11'], '10': ['11', '00']}, '63|88': {'01': ['11'], '10': ['11', '00']}, '64|91': {'01': ['11'], '10': ['11', '00']}, '65|94': {'01': ['11'], '10': ['11', '00']}, '66|97': {'01': ['11'], '10': ['11', '00']}, '60|78': {'01': ['11'], '10': ['11', '00']}, '61|81': {'01': ['11'], '10': ['11', '00']}, '62|84': {'01': ['11'], '10': ['11', '00']}, '63|87': {'01': ['11'], '10': ['11', '00']}, '64|90': {'01': ['11'], '10': ['11', '00']}, '65|93': {'01': ['11'], '10': ['11', '00']}, '66|96': {'01': ['11'], '10': ['11', '00']}, '67|99': {'01': ['11'], '10': ['11', '00']}, '60|77': {'01': ['11'], '10': ['11', '00']}, '61|80': {'01': ['11'], '10': ['11', '00']}, '62|83': {'01': ['11'], '10': ['11', '00']}, '63|86': {'01': ['11'], '10': ['11', '00']}, '64|89': {'01': ['11'], '10': ['11', '00']}, '65|92': {'01': ['11'], '10': ['11', '00']}, '66|95': {'01': ['11'], '10': ['11', '00']}, '67|98': {'01': ['11'], '10': ['11', '00']}, '60|76': {'01': ['11'], '10': ['11', '00']}, '61|79': {'01': ['11'], '10': ['11', '00']}, '62|82': {'01': ['11'], '10': ['11', '00']}, '63|85': {'01': ['11'], '10': ['11', '00']}, '64|88': {'01': ['11'], '10': ['11', '00']}, '65|91': {'01': ['11'], '10': ['11', '00']}, '66|94': {'01': ['11'], '10': ['11', '00']}, '67|97': {'01': ['11'], '10': ['11', '00']}, '60|75': {'01': ['11'], '10': ['11', '00']}, '61|78': {'01': ['11'], '10': ['11', '00']}, '62|81': {'01': ['11'], '10': ['11', '00']}, '63|84': {'01': ['11'], '10': ['11', '00']}, '64|87': {'01': ['11'], '10': ['11', '00']}, '65|90': {'01': ['11'], '10': ['11', '00']}, '66|93': {'01': ['11'], '10': ['11', '00']}, '67|96': {'01': ['11'], '10': ['11', '00']}, '68|99': {'01': ['11'], '10': ['11', '00']}, '60|74': {'01': ['11'], '10': ['11', '00']}, '61|77': {'01': ['11'], '10': ['11', '00']}, '62|80': {'01': ['11'], '10': ['11', '00']}, '63|83': {'01': ['11'], '10': ['11', '00']}, '64|86': {'01': ['11'], '10': ['11', '00']}, '65|89': {'01': ['11'], '10': ['11', '00']}, '66|92': {'01': ['11'], '10': ['11', '00']}, '67|95': {'01': ['11'], '10': ['11', '00']}, '68|98': {'01': ['11'], '10': ['11', '00']}, '60|73': {'01': ['11'], '10': ['11', '00']}, '61|76': {'01': ['11'], '10': ['11', '00']}, '62|79': {'01': ['11'], '10': ['11', '00']}, '63|82': {'01': ['11'], '10': ['11', '00']}, '64|85': {'01': ['11'], '10': ['11', '00']}, '65|88': {'01': ['11'], '10': ['11', '00']}, '66|91': {'01': ['11'], '10': ['11', '00']}, '67|94': {'01': ['11'], '10': ['11', '00']}, '68|97': {'01': ['11'], '10': ['11', '00']}, '60|72': {'01': ['11'], '10': ['11', '00']}, '61|75': {'01': ['11'], '10': ['11', '00']}, '62|78': {'01': ['11'], '10': ['11', '00']}, '63|81': {'01': ['11'], '10': ['11', '00']}, '64|84': {'01': ['11'], '10': ['11', '00']}, '65|87': {'01': ['11'], '10': ['11', '00']}, '66|90': {'01': ['11'], '10': ['11', '00']}, '67|93': {'01': ['11'], '10': ['11', '00']}, '68|96': {'01': ['11'], '10': ['11', '00']}, '69|99': {'01': ['11'], '10': ['11', '00']}, '60|71': {'01': ['11'], '10': ['11', '00']}, '61|74': {'01': ['11'], '10': ['11', '00']}, '62|77': {'01': ['11'], '10': ['11', '00']}, '63|80': {'01': ['11'], '10': ['11', '00']}, '64|83': {'01': ['11'], '10': ['11', '00']}, '65|86': {'01': ['11'], '10': ['11', '00']}, '66|89': {'01': ['11'], '10': ['11', '00']}, '67|92': {'01': ['11'], '10': ['11', '00']}, '68|95': {'01': ['11'], '10': ['11', '00']}, '69|98': {'01': ['11'], '10': ['11', '00']}, '60|70': {'01': ['11'], '10': ['11', '00']}, '61|73': {'01': ['11'], '10': ['11', '00']}, '62|76': {'01': ['11'], '10': ['11', '00']}, '63|79': {'01': ['11'], '10': ['11', '00']}, '64|82': {'01': ['11'], '10': ['11', '00']}, '65|85': {'01': ['11'], '10': ['11', '00']}, '66|88': {'01': ['11'], '10': ['11', '00']}, '67|91': {'01': ['11'], '10': ['11', '00']}, '68|94': {'01': ['11'], '10': ['11', '00']}, '69|97': {'01': ['11'], '10': ['11', '00']}, '60|69': {'01': ['11'], '10': ['11', '00']}, '61|72': {'01': ['11'], '10': ['11', '00']}, '62|75': {'01': ['11'], '10': ['11', '00']}, '63|78': {'01': ['11'], '10': ['11', '00']}, '64|81': {'01': ['11'], '10': ['11', '00']}, '65|84': {'01': ['11'], '10': ['11', '00']}, '66|87': {'01': ['11'], '10': ['11', '00']}, '67|90': {'01': ['11'], '10': ['11', '00']}, '68|93': {'01': ['11'], '10': ['11', '00']}, '69|96': {'01': ['11'], '10': ['11', '00']}, '70|99': {'01': ['11'], '10': ['11', '00']}, '60|68': {'01': ['11'], '10': ['11', '00']}, '61|71': {'01': ['11'], '10': ['11', '00']}, '62|74': {'01': ['11'], '10': ['11', '00']}, '63|77': {'01': ['11'], '10': ['11', '00']}, '64|80': {'01': ['11'], '10': ['11', '00']}, '65|83': {'01': ['11'], '10': ['11', '00']}, '66|86': {'01': ['11'], '10': ['11', '00']}, '67|89': {'01': ['11'], '10': ['11', '00']}, '68|92': {'01': ['11'], '10': ['11', '00']}, '69|95': {'01': ['11'], '10': ['11', '00']}, '70|98': {'01': ['11'], '10': ['11', '00']}, '60|67': {'01': ['11'], '10': ['11', '00']}, '61|70': {'01': ['11'], '10': ['11', '00']}, '62|73': {'01': ['11'], '10': ['11', '00']}, '63|76': {'01': ['11'], '10': ['11', '00']}, '64|79': {'01': ['11'], '10': ['11', '00']}, '65|82': {'01': ['11'], '10': ['11', '00']}, '66|85': {'01': ['11'], '10': ['11', '00']}, '67|88': {'01': ['11'], '10': ['11', '00']}, '68|91': {'01': ['11'], '10': ['11', '00']}, '69|94': {'01': ['11'], '10': ['11', '00']}, '70|97': {'01': ['11'], '10': ['11', '00']}, '59|64': {'01': ['11'], '10': ['11', '00']}, '60|66': {'01': ['11'], '10': ['11', '00']}, '61|69': {'01': ['11'], '10': ['11', '00']}, '62|72': {'01': ['11'], '10': ['11', '00']}, '63|75': {'01': ['11'], '10': ['11', '00']}, '64|78': {'01': ['11'], '10': ['11', '00']}, '65|81': {'01': ['11'], '10': ['11', '00']}, '66|84': {'01': ['11'], '10': ['11', '00']}, '67|87': {'01': ['11'], '10': ['11', '00']}, '68|90': {'01': ['11'], '10': ['11', '00']}, '69|93': {'01': ['11'], '10': ['11', '00']}, '70|96': {'01': ['11'], '10': ['11', '00']}, '71|99': {'01': ['11'], '10': ['11', '00']}, '59|63': {'01': ['11'], '10': ['11', '00']}, '60|65': {'01': ['11'], '10': ['11', '00']}, '61|68': {'01': ['11'], '10': ['11', '00']}, '62|71': {'01': ['11'], '10': ['11', '00']}, '63|74': {'01': ['11'], '10': ['11', '00']}, '64|77': {'01': ['11'], '10': ['11', '00']}, '65|80': {'01': ['11'], '10': ['11', '00']}, '66|83': {'01': ['11'], '10': ['11', '00']}, '67|86': {'01': ['11'], '10': ['11', '00']}, '68|89': {'01': ['11'], '10': ['11', '00']}, '69|92': {'01': ['11'], '10': ['11', '00']}, '70|95': {'01': ['11'], '10': ['11', '00']}, '71|98': {'01': ['11'], '10': ['11', '00']}, '59|62': {'01': ['11'], '10': ['11', '00']}, '60|64': {'01': ['11'], '10': ['11', '00']}, '61|67': {'01': ['11'], '10': ['11', '00']}, '62|70': {'01': ['11'], '10': ['11', '00']}, '63|73': {'01': ['11'], '10': ['11', '00']}, '64|76': {'01': ['11'], '10': ['11', '00']}, '65|79': {'01': ['11'], '10': ['11', '00']}, '66|82': {'01': ['11'], '10': ['11', '00']}, '67|85': {'01': ['11'], '10': ['11', '00']}, '68|88': {'01': ['11'], '10': ['11', '00']}, '69|91': {'01': ['11'], '10': ['11', '00']}, '70|94': {'01': ['11'], '10': ['11', '00']}, '71|97': {'01': ['11'], '10': ['11', '00']}, '59|61': {'01': ['11'], '10': ['11', '00']}, '60|63': {'01': ['11'], '10': ['11', '00']}, '61|66': {'01': ['11'], '10': ['11', '00']}, '62|69': {'01': ['11'], '10': ['11', '00']}, '63|72': {'01': ['11'], '10': ['11', '00']}, '64|75': {'01': ['11'], '10': ['11', '00']}, '65|78': {'01': ['11'], '10': ['11', '00']}, '66|81': {'01': ['11'], '10': ['11', '00']}, '67|84': {'01': ['11'], '10': ['11', '00']}, '68|87': {'01': ['11'], '10': ['11', '00']}, '69|90': {'01': ['11'], '10': ['11', '00']}, '70|93': {'01': ['11'], '10': ['11', '00']}, '71|96': {'01': ['11'], '10': ['11', '00']}, '72|99': {'01': ['11'], '10': ['11', '00']}, '59|60': {'01': ['11'], '10': ['11', '00']}, '59|59': {'01': ['11'], '10': ['11', '00']}, '60|62': {'01': ['11'], '10': ['11', '00']}, '61|65': {'01': ['11'], '10': ['11', '00']}, '62|68': {'01': ['11'], '10': ['11', '00']}, '63|71': {'01': ['11'], '10': ['11', '00']}, '64|74': {'01': ['11'], '10': ['11', '00']}, '65|77': {'01': ['11'], '10': ['11', '00']}, '66|80': {'01': ['11'], '10': ['11', '00']}, '67|83': {'01': ['11'], '10': ['11', '00']}, '68|86': {'01': ['11'], '10': ['11', '00']}, '69|89': {'01': ['11'], '10': ['11', '00']}, '70|92': {'01': ['11'], '10': ['11', '00']}, '71|95': {'01': ['11'], '10': ['11', '00']}, '72|98': {'01': ['11'], '10': ['11', '00']}, '56|53': {'01': ['11'], '10': ['11', '00']}, '58|56': {'01': ['11'], '10': ['11', '00']}, '59|58': {'01': ['11'], '10': ['11', '00']}, '60|61': {'01': ['11'], '10': ['11', '00']}, '61|64': {'01': ['11'], '10': ['11', '00']}, '62|67': {'01': ['11'], '10': ['11', '00']}, '63|70': {'01': ['11'], '10': ['11', '00']}, '64|73': {'01': ['11'], '10': ['11', '00']}, '65|76': {'01': ['11'], '10': ['11', '00']}, '66|79': {'01': ['11'], '10': ['11', '00']}, '67|82': {'01': ['11'], '10': ['11', '00']}, '68|85': {'01': ['11'], '10': ['11', '00']}, '69|88': {'01': ['11'], '10': ['11', '00']}, '70|91': {'01': ['11'], '10': ['11', '00']}, '71|94': {'01': ['11'], '10': ['11', '00']}, '72|97': {'01': ['11'], '10': ['11', '00']}, '57|54': {'01': ['11'], '10': ['11', '00']}, '54|49': {'00': ['11'], '10': ['11', '01']}, '54|50': {'10': ['00', '11']}, '55|51': {'01': ['11'], '10': ['11', '00']}, '55|50': {'01': ['11'], '10': ['00', '11']}, '56|52': {'01': ['11'], '10': ['11', '00']}, '58|55': {'01': ['11'], '10': ['11', '00']}, '59|57': {'01': ['11'], '10': ['11', '00']}, '60|60': {'01': ['11'], '10': ['11', '00']}, '61|63': {'01': ['11'], '10': ['11', '00']}, '62|66': {'01': ['11'], '10': ['11', '00']}, '63|69': {'01': ['11'], '10': ['11', '00']}, '64|72': {'01': ['11'], '10': ['11', '00']}, '65|75': {'01': ['11'], '10': ['11', '00']}, '66|78': {'01': ['11'], '10': ['11', '00']}, '67|81': {'01': ['11'], '10': ['11', '00']}, '68|84': {'01': ['11'], '10': ['11', '00']}, '69|87': {'01': ['11'], '10': ['11', '00']}, '70|90': {'01': ['11'], '10': ['11', '00']}, '71|93': {'01': ['11'], '10': ['11', '00']}, '72|96': {'01': ['11'], '10': ['11', '00']}, '73|99': {'01': ['11'], '10': ['11', '00']}, '54|48': {'00': ['11'], '10': ['11', '01']}, '54|47': {'00': ['11'], '10': ['11', '01']}, '53|45': {'00': ['11'], '10': ['11', '01']}, '52|42': {'00': ['11'], '10': ['01', '11']}, '53|44': {'00': ['11'], '10': ['01', '11']}, '54|46': {'00': ['11'], '10': ['01', '11']}, '55|49': {'00': ['11'], '10': ['11']}, '56|51': {'01': ['11'], '10': ['11', '00']}, '57|53': {'01': ['11'], '10': ['11', '00']}, '56|50': {'01': ['11'], '10': ['00', '11']}, '57|52': {'01': ['11'], '10': ['11', '00']}, '58|54': {'01': ['11'], '10': ['11', '00']}, '59|56': {'01': ['11'], '10': ['11', '00']}, '60|59': {'01': ['11'], '10': ['11', '00']}, '61|62': {'01': ['11'], '10': ['11', '00']}, '62|65': {'01': ['11'], '10': ['11', '00']}, '63|68': {'01': ['11'], '10': ['11', '00']}, '64|71': {'01': ['11'], '10': ['11', '00']}, '65|74': {'01': ['11'], '10': ['11', '00']}, '66|77': {'01': ['11'], '10': ['11', '00']}, '67|80': {'01': ['11'], '10': ['11', '00']}, '68|83': {'01': ['11'], '10': ['11', '00']}, '69|86': {'01': ['11'], '10': ['11', '00']}, '70|89': {'01': ['11'], '10': ['11', '00']}, '71|92': {'01': ['11'], '10': ['11', '00']}, '72|95': {'01': ['11'], '10': ['11', '00']}, '73|98': {'01': ['11'], '10': ['11', '00']}, '55|48': {'00': ['11'], '10': ['01', '11']}, '51|39': {'00': ['11'], '10': ['11', '01']}, '52|41': {'00': ['11'], '10': ['01', '11']}, '53|43': {'00': ['11'], '10': ['11', '01']}, '54|45': {'00': ['11'], '10': ['01', '11']}, '55|47': {'00': ['11'], '10': ['11', '01']}, '56|48': {'00': ['11'], '10': ['11', '01']}, '56|49': {'10': ['11']}, '57|51': {'01': ['11'], '10': ['11', '00']}, '58|53': {'01': ['11'], '10': ['11', '00']}, '59|55': {'01': ['11'], '10': ['11', '00']}, '60|58': {'01': ['11'], '10': ['11', '00']}, '61|61': {'01': ['11'], '10': ['11', '00']}, '62|64': {'01': ['11'], '10': ['11', '00']}, '63|67': {'01': ['11'], '10': ['11', '00']}, '64|70': {'01': ['11'], '10': ['11', '00']}, '65|73': {'01': ['11'], '10': ['11', '00']}, '66|76': {'01': ['11'], '10': ['11', '00']}, '67|79': {'01': ['11'], '10': ['11', '00']}, '68|82': {'01': ['11'], '10': ['11', '00']}, '69|85': {'01': ['11'], '10': ['11', '00']}, '70|88': {'01': ['11'], '10': ['11', '00']}, '71|91': {'01': ['11'], '10': ['11', '00']}, '72|94': {'01': ['11'], '10': ['11', '00']}, '73|97': {'01': ['11'], '10': ['11', '00']}, '57|50': {'01': ['11'], '10': ['00', '11']}, '55|46': {'00': ['11'], '10': ['01', '11']}, '54|44': {'00': ['11'], '10': ['01', '11']}, '56|47': {'00': ['11'], '10': ['11', '01']}, '55|45': {'00': ['11'], '10': ['01', '11']}, '56|46': {'00': ['11'], '10': ['01', '11']}, '57|48': {'00': ['11'], '10': ['11', '01']}, '57|49': {'10': ['11']}, '58|52': {'01': ['11'], '10': ['11', '00']}, '58|51': {'01': ['11'], '10': ['11', '00']}, '59|54': {'01': ['11'], '10': ['11', '00']}, '60|57': {'01': ['11'], '10': ['11', '00']}, '61|60': {'01': ['11'], '10': ['11', '00']}, '62|63': {'01': ['11'], '10': ['11', '00']}, '63|66': {'01': ['11'], '10': ['11', '00']}, '64|69': {'01': ['11'], '10': ['11', '00']}, '65|72': {'01': ['11'], '10': ['11', '00']}, '66|75': {'01': ['11'], '10': ['11', '00']}, '67|78': {'01': ['11'], '10': ['11', '00']}, '68|81': {'01': ['11'], '10': ['11', '00']}, '69|84': {'01': ['11'], '10': ['11', '00']}, '70|87': {'01': ['11'], '10': ['11', '00']}, '71|90': {'01': ['11'], '10': ['11', '00']}, '72|93': {'01': ['11'], '10': ['11', '00']}, '73|96': {'01': ['11'], '10': ['11', '00']}, '74|99': {'01': ['11'], '10': ['11', '00']}, '58|50': {'01': ['11'], '10': ['00', '11']}, '59|53': {'01': ['11'], '10': ['11', '00']}, '60|56': {'01': ['11'], '10': ['11', '00']}, '61|59': {'01': ['11'], '10': ['11', '00']}, '62|62': {'01': ['11'], '10': ['11', '00']}, '63|65': {'01': ['11'], '10': ['11', '00']}, '64|68': {'01': ['11'], '10': ['11', '00']}, '65|71': {'01': ['11'], '10': ['11', '00']}, '66|74': {'01': ['11'], '10': ['11', '00']}, '67|77': {'01': ['11'], '10': ['11', '00']}, '68|80': {'01': ['11'], '10': ['11', '00']}, '69|83': {'01': ['11'], '10': ['11', '00']}, '70|86': {'01': ['11'], '10': ['11', '00']}, '71|89': {'01': ['11'], '10': ['11', '00']}, '72|92': {'01': ['11'], '10': ['11', '00']}, '73|95': {'01': ['11'], '10': ['11', '00']}, '74|98': {'01': ['11'], '10': ['11', '00']}, '53|42': {'00': ['11'], '10': ['11', '01']}, '50|36': {'00': ['11'], '10': ['11', '01']}, '51|38': {'00': ['11'], '10': ['01', '11']}, '52|40': {'00': ['11'], '10': ['01', '11']}, '54|43': {'00': ['11'], '10': ['01', '11']}, '55|44': {'00': ['11'], '10': ['01', '11']}, '56|45': {'00': ['11'], '10': ['01', '11']}, '57|47': {'00': ['11'], '10': ['01', '11']}, '57|46': {'00': ['11'], '10': ['01', '11']}, '58|48': {'00': ['11'], '10': ['11', '01']}, '58|49': {'10': ['11']}, '59|52': {'01': ['11'], '10': ['11', '00']}, '60|55': {'01': ['11'], '10': ['11', '00']}, '61|58': {'01': ['11'], '10': ['11', '00']}, '62|61': {'01': ['11'], '10': ['11', '00']}, '63|64': {'01': ['11'], '10': ['11', '00']}, '64|67': {'01': ['11'], '10': ['11', '00']}, '65|70': {'01': ['11'], '10': ['11', '00']}, '66|73': {'01': ['11'], '10': ['11', '00']}, '67|76': {'01': ['11'], '10': ['11', '00']}, '68|79': {'01': ['11'], '10': ['11', '00']}, '69|82': {'01': ['11'], '10': ['11', '00']}, '70|85': {'01': ['11'], '10': ['11', '00']}, '71|88': {'01': ['11'], '10': ['11', '00']}, '72|91': {'01': ['11'], '10': ['11', '00']}, '73|94': {'01': ['11'], '10': ['11', '00']}, '74|97': {'01': ['11'], '10': ['11', '00']}, '59|51': {'01': ['11'], '10': ['11', '00']}, '60|54': {'01': ['11'], '10': ['11', '00']}, '61|57': {'01': ['11'], '10': ['11', '00']}, '62|60': {'01': ['11'], '10': ['11', '00']}, '63|63': {'01': ['11'], '10': ['11', '00']}, '64|66': {'01': ['11'], '10': ['11', '00']}, '65|69': {'01': ['11'], '10': ['11', '00']}, '66|72': {'01': ['11'], '10': ['11', '00']}, '67|75': {'01': ['11'], '10': ['11', '00']}, '68|78': {'01': ['11'], '10': ['11', '00']}, '69|81': {'01': ['11'], '10': ['11', '00']}, '70|84': {'01': ['11'], '10': ['11', '00']}, '71|87': {'01': ['11'], '10': ['11', '00']}, '72|90': {'01': ['11'], '10': ['11', '00']}, '73|93': {'01': ['11'], '10': ['11', '00']}, '74|96': {'01': ['11'], '10': ['11', '00']}, '75|99': {'01': ['11'], '10': ['11', '00']}, '59|50': {'01': ['11'], '10': ['00', '11']}, '60|53': {'01': ['11'], '10': ['11', '00']}, '61|56': {'01': ['11'], '10': ['11', '00']}, '62|59': {'01': ['11'], '10': ['11', '00']}, '63|62': {'01': ['11'], '10': ['11', '00']}, '64|65': {'01': ['11'], '10': ['11', '00']}, '65|68': {'01': ['11'], '10': ['11', '00']}, '66|71': {'01': ['11'], '10': ['11', '00']}, '67|74': {'01': ['11'], '10': ['11', '00']}, '68|77': {'01': ['11'], '10': ['11', '00']}, '69|80': {'01': ['11'], '10': ['11', '00']}, '70|83': {'01': ['11'], '10': ['11', '00']}, '71|86': {'01': ['11'], '10': ['11', '00']}, '72|89': {'01': ['11'], '10': ['11', '00']}, '73|92': {'01': ['11'], '10': ['11', '00']}, '74|95': {'01': ['11'], '10': ['11', '00']}, '75|98': {'01': ['11'], '10': ['11', '00']}, '58|47': {'00': ['11'], '10': ['01', '11']}, '53|41': {'00': ['11'], '10': ['11', '01']}, '50|35': {'00': ['11'], '10': ['11', '01']}, '49|33': {'00': ['11'], '10': ['11', '01']}, '51|37': {'00': ['11'], '10': ['01', '11']}, '52|39': {'00': ['11'], '10': ['01', '11']}, '54|42': {'00': ['11'], '10': ['01', '11']}, '55|43': {'00': ['11'], '10': ['01', '11']}, '56|44': {'00': ['11'], '10': ['01', '11']}, '57|45': {'00': ['01', '11'], '10': ['01']}, '58|46': {'00': ['01', '11'], '10': ['01'], '11': []}, '59|48': {'00': ['11'], '10': ['01', '11']}, '59|49': {'10': ['11']}, '60|52': {'01': ['11'], '10': ['11', '00']}, '61|55': {'01': ['11'], '10': ['11', '00']}, '62|58': {'01': ['11'], '10': ['11', '00']}, '63|61': {'01': ['11'], '10': ['11', '00']}, '64|64': {'01': ['11'], '10': ['11', '00']}, '65|67': {'01': ['11'], '10': ['11', '00']}, '66|70': {'01': ['11'], '10': ['11', '00']}, '67|73': {'01': ['11'], '10': ['11', '00']}, '68|76': {'01': ['11'], '10': ['11', '00']}, '69|79': {'01': ['11'], '10': ['11', '00']}, '70|82': {'01': ['11'], '10': ['11', '00']}, '71|85': {'01': ['11'], '10': ['11', '00']}, '72|88': {'01': ['11'], '10': ['11', '00']}, '73|91': {'01': ['11'], '10': ['11', '00']}, '74|94': {'01': ['11'], '10': ['11', '00']}, '75|97': {'01': ['11'], '10': ['11', '00']}, '60|51': {'01': ['11'], '10': ['11', '00']}, '61|54': {'01': ['11'], '10': ['11', '00']}, '62|57': {'01': ['11'], '10': ['11', '00']}, '63|60': {'01': ['11'], '10': ['11', '00']}, '64|63': {'01': ['11'], '10': ['11', '00']}, '65|66': {'01': ['11'], '10': ['11', '00']}, '66|69': {'01': ['11'], '10': ['11', '00']}, '67|72': {'01': ['11'], '10': ['11', '00']}, '68|75': {'01': ['11'], '10': ['11', '00']}, '69|78': {'01': ['11'], '10': ['11', '00']}, '70|81': {'01': ['11'], '10': ['11', '00']}, '71|84': {'01': ['11'], '10': ['11', '00']}, '72|87': {'01': ['11'], '10': ['11', '00']}, '73|90': {'01': ['11'], '10': ['11', '00']}, '74|93': {'01': ['11'], '10': ['11', '00']}, '75|96': {'01': ['11'], '10': ['11', '00']}, '76|99': {'01': ['11'], '10': ['11', '00']}, '60|50': {'01': ['11'], '10': ['00', '11']}, '61|53': {'01': ['11'], '10': ['11', '00']}, '62|56': {'01': ['11'], '10': ['11', '00']}, '63|59': {'01': ['11'], '10': ['11', '00']}, '64|62': {'01': ['11'], '10': ['11', '00']}, '65|65': {'01': ['11'], '10': ['11', '00']}, '66|68': {'01': ['11'], '10': ['11', '00']}, '67|71': {'01': ['11'], '10': ['11', '00']}, '68|74': {'01': ['11'], '10': ['11', '00']}, '69|77': {'01': ['11'], '10': ['11', '00']}, '70|80': {'01': ['11'], '10': ['11', '00']}, '71|83': {'01': ['11'], '10': ['11', '00']}, '72|86': {'01': ['11'], '10': ['11', '00']}, '73|89': {'01': ['11'], '10': ['11', '00']}, '74|92': {'01': ['11'], '10': ['11', '00']}, '75|95': {'01': ['11'], '10': ['11', '00']}, '76|98': {'01': ['11'], '10': ['11', '00']}, '59|47': {'00': ['11'], '10': ['01', '11'], '11': []}, '60|48': {'00': ['11'], '10': ['01', '11'], '11': []}, '60|49': {'10': ['11']}, '61|52': {'01': ['11'], '10': ['11', '00']}, '62|55': {'01': ['11'], '10': ['11', '00']}, '63|58': {'01': ['11'], '10': ['11', '00']}, '64|61': {'01': ['11'], '10': ['11', '00']}, '65|64': {'01': ['11'], '10': ['11', '00']}, '66|67': {'01': ['11'], '10': ['11', '00']}, '67|70': {'01': ['11'], '10': ['11', '00']}, '68|73': {'01': ['11'], '10': ['11', '00']}, '69|76': {'01': ['11'], '10': ['11', '00']}, '70|79': {'01': ['11'], '10': ['11', '00']}, '71|82': {'01': ['11'], '10': ['11', '00']}, '72|85': {'01': ['11'], '10': ['11', '00']}, '73|88': {'01': ['11'], '10': ['11', '00']}, '74|91': {'01': ['11'], '10': ['11', '00']}, '75|94': {'01': ['11'], '10': ['11', '00']}, '76|97': {'01': ['11'], '10': ['11', '00']}, '61|51': {'01': ['11'], '10': ['11', '00']}, '62|54': {'01': ['11'], '10': ['11', '00']}, '63|57': {'01': ['11'], '10': ['11', '00']}, '64|60': {'01': ['11'], '10': ['11', '00']}, '65|63': {'01': ['11'], '10': ['11', '00']}, '66|66': {'01': ['11'], '10': ['11', '00']}, '67|69': {'01': ['11'], '10': ['11', '00']}, '68|72': {'01': ['11'], '10': ['11', '00']}, '69|75': {'01': ['11'], '10': ['11', '00']}, '70|78': {'01': ['11'], '10': ['11', '00']}, '71|81': {'01': ['11'], '10': ['11', '00']}, '72|84': {'01': ['11'], '10': ['11', '00']}, '73|87': {'01': ['11'], '10': ['11', '00']}, '74|90': {'01': ['11'], '10': ['11', '00']}, '75|93': {'01': ['11'], '10': ['11', '00']}, '76|96': {'01': ['11'], '10': ['11', '00']}, '77|99': {'01': ['11'], '10': ['11', '00']}, '61|50': {'01': ['11'], '10': ['00', '11']}, '62|53': {'01': ['11'], '10': ['11', '00']}, '63|56': {'01': ['11'], '10': ['11', '00']}, '64|59': {'01': ['11'], '10': ['11', '00']}, '65|62': {'01': ['11'], '10': ['11', '00']}, '66|65': {'01': ['11'], '10': ['11', '00']}, '67|68': {'01': ['11'], '10': ['11', '00']}, '68|71': {'01': ['11'], '10': ['11', '00']}, '69|74': {'01': ['11'], '10': ['11', '00']}, '70|77': {'01': ['11'], '10': ['11', '00']}, '71|80': {'01': ['11'], '10': ['11', '00']}, '72|83': {'01': ['11'], '10': ['11', '00']}, '73|86': {'01': ['11'], '10': ['11', '00']}, '74|89': {'01': ['11'], '10': ['11', '00']}, '75|92': {'01': ['11'], '10': ['11', '00']}, '76|95': {'01': ['11'], '10': ['11', '00']}, '77|98': {'01': ['11'], '10': ['11', '00']}, '59|46': {'00': ['01', '11', '10'], '11': []}, '60|47': {'00': ['01', '10', '11'], '10': ['10'], '11': []}, '61|48': {'00': ['11'], '11': []}, '61|49': {'10': ['11']}, '62|52': {'01': ['11'], '10': ['11']}, '63|55': {'01': ['11'], '10': ['11']}, '64|58': {'01': ['11'], '10': ['11']}, '65|61': {'01': ['11'], '10': ['11']}, '66|64': {'01': ['11'], '10': ['11']}, '67|67': {'01': ['11'], '10': ['11']}, '68|70': {'01': ['11'], '10': ['11']}, '69|73': {'01': ['11'], '10': ['11']}, '70|76': {'01': ['11'], '10': ['11']}, '71|79': {'01': ['11'], '10': ['11']}, '72|82': {'01': ['11'], '10': ['11']}, '73|85': {'01': ['11'], '10': ['11']}, '74|88': {'01': ['11'], '10': ['11']}, '75|91': {'01': ['11'], '10': ['11']}, '76|94': {'01': ['11'], '10': ['11']}, '77|97': {'01': ['11'], '10': ['11']}, '62|51': {'01': ['11'], '10': ['11']}, '63|54': {'01': ['11'], '10': ['11']}, '64|57': {'01': ['11'], '10': ['11']}, '65|60': {'01': ['11'], '10': ['11']}, '66|63': {'01': ['11'], '10': ['11']}, '67|66': {'01': ['11'], '10': ['11']}, '68|69': {'01': ['11'], '10': ['11']}, '69|72': {'01': ['11'], '10': ['11']}, '70|75': {'01': ['11'], '10': ['11']}, '71|78': {'01': ['11'], '10': ['11']}, '72|81': {'01': ['11'], '10': ['11']}, '73|84': {'01': ['11'], '10': ['11']}, '74|87': {'01': ['11'], '10': ['11']}, '75|90': {'01': ['11'], '10': ['11']}, '76|93': {'01': ['11'], '10': ['11']}, '77|96': {'01': ['11'], '10': ['11']}, '78|99': {'01': ['11'], '10': ['11']}, '62|50': {'01': ['11']}, '63|53': {'01': ['11']}, '64|56': {'01': ['11']}, '65|59': {'01': ['11']}, '66|62': {'01': ['11']}, '67|65': {'01': ['11']}, '68|68': {'01': ['11']}, '69|71': {'01': ['11']}, '70|74': {'01': ['11']}, '71|77': {'01': ['11']}, '72|80': {'01': ['11']}, '73|83': {'01': ['11']}, '74|86': {'01': ['11']}, '75|89': {'01': ['11']}, '76|92': {'01': ['11']}, '77|95': {'01': ['11']}, '78|98': {'01': ['11']}, '61|47': {'00': ['01', '10'], '11': []}, '62|47': {'00': ['10'], '11': ['01']}, '62|48': {'11': []}, '60|46': {'00': ['01', '10', '11'], '11': ['01']}, '61|46': {'00': ['10'], '11': ['01', '10']}, '58|45': {'00': ['11', '01'], '10': ['01', '10'], '11': []}, '53|40': {'00': ['11'], '10': ['11', '01']}, '52|38': {'00': ['11'], '10': ['01', '11']}, '54|41': {'00': ['11'], '10': ['01', '11']}, '55|42': {'00': ['11'], '10': ['01', '11']}, '56|43': {'00': ['01', '11'], '10': ['01']}, '57|44': {'00': ['01', '11'], '10': ['01']}, '59|45': {'00': ['01', '10'], '11': ['01']}, '60|45': {'00': ['10'], '11': ['01', '10']}, '58|44': {'00': ['01', '11', '10'], '11': ['01']}, '59|44': {'00': ['10'], '11': ['01']}, '57|43': {'00': ['11', '01'], '10': ['01']}, '53|39': {'00': ['11'], '10': ['11', '01']}, '50|34': {'00': ['11'], '10': ['11', '01']}, '49|32': {'00': ['11'], '10': ['11', '01']}, '51|36': {'00': ['11'], '10': ['01', '11']}, '52|37': {'00': ['11'], '10': ['01', '11']}, '54|40': {'00': ['11'], '10': ['01', '11']}, '55|41': {'00': ['11'], '10': ['01', '11']}, '56|42': {'00': ['01', '11'], '10': ['01']}, '58|43': {'00': ['01', '10'], '11': ['01']}, '59|43': {'00': ['10'], '11': ['01', '10']}, '60|44': {'00': ['10'], '11': ['01', '10']}, '61|45': {'00': ['10'], '11': ['10', '01']}, '62|46': {'00': ['10'], '11': ['10']}, '57|42': {'00': ['01', '11'], '11': ['01']}, '58|42': {'00': ['01', '10'], '11': ['01']}, '59|42': {'00': ['10'], '11': ['01', '10']}, '60|43': {'00': ['10'], '11': ['10', '01']}, '61|44': {'00': ['10'], '11': ['10']}, '63|47': {'00': ['10']}, '63|48': {'11': []}, '53|38': {'00': ['11'], '10': ['11', '01']}, '48|30': {'00': ['11'], '10': ['01', '11']}, '50|33': {'00': ['11'], '10': ['11', '01']}, '49|31': {'00': ['11'], '10': ['11', '01']}, '51|35': {'00': ['11'], '10': ['01', '11']}, '52|36': {'00': ['11'], '10': ['01', '11']}, '54|39': {'00': ['11'], '10': ['01', '11']}, '55|40': {'00': ['11'], '10': ['01', '11']}, '56|41': {'00': ['01', '11'], '10': ['01']}, '57|41': {'00': ['01', '10'], '11': []}, '58|41': {'00': ['01', '10'], '11': ['01']}, '59|41': {'00': ['10'], '11': ['01', '10']}, '60|42': {'00': ['10'], '11': ['10']}, '53|37': {'00': ['11'], '10': ['11', '01']}, '48|29': {'00': ['11'], '10': ['01', '11']}, '50|32': {'00': ['11'], '10': ['11']}, '49|30': {'00': ['11'], '10': ['11']}, '51|34': {'00': ['11'], '10': ['01', '11']}, '52|35': {'00': ['11'], '10': ['01', '11']}, '54|38': {'00': ['11'], '10': ['01', '11']}, '55|39': {'00': ['01', '11'], '10': ['01']}, '56|40': {'00': ['01', '11'], '10': ['01']}, '57|40': {'00': ['01', '10'], '11': ['01']}, '58|40': {'00': ['01', '10'], '11': ['01']}, '59|40': {'00': ['10']}, '60|41': {'00': ['10']}, '61|43': {'00': ['10']}, '62|45': {'00': ['10']}, '56|39': {'00': ['11', '01'], '10': ['01']}, '53|36': {'00': ['11'], '10': ['11']}, '45|25': {'00': ['11']}, '47|27': {'00': ['11']}, '48|28': {'00': ['11']}, '50|31': {'00': ['11']}, '49|29': {'00': ['11']}, '51|33': {'00': ['11']}, '52|34': {'00': ['11']}, '54|37': {'00': ['11']}, '55|38': {'00': ['01', '11']}, '57|39': {'00': ['01', '10']}, '58|39': {'00': ['10']}, '56|38': {'00': ['11']}, '53|35': {'00': ['11']}}

#tSSC = filter_with_entry_exit_pairs( intersect_FB(visitedF, visitedB) )
#write_out_inter(tSSC, "IntersectionForwardBackward" )
#print( tSSC )
